# Project Overview

> This is an in-depth notebook which explores the King County Housing Dataset through several different models. The notebook includes a thorough EDA and cleaning section, study of different models using different categorical methods (one-hot encoding vs target encoding) with extensive parameter tuning, an exploration of different feature selection methods, an evaluation of the final model, and visualizations.

* **Business Objective**


* **Notebook Preparation**
    * Importing our Modules


* **Preprocessing**
    * EDA and Cleaning
        * Scaling Time Series
        * Duplicates
        * Outlier Detection
        * Missing Data
        * Binary Data
        * Visualize Cleaned Data
        * Studying our Target Variable
    * Create Holdout Set
    * Feature Engineering
    * Correlations and Multicollinearity
    * EDA & Process Train Set
        * Categoricals
        * Continuous
            * Find Interactions
            * Transform and Standardize Continuous Data
            * Add Polynomial Features
    * Process Test Set
        * Categoricals
        * Continuous
    * Create Train/Test Split


* **Model Explorations**
    * Picking our Base Features
    * Linear Regressions
        * Basic LR with Top Features One-Hot Encoded
        * Basic LR with Top Features Target Encoded
        * LR with ALL model features
        * Linear Regression with various Feature Selection Methods
            * Forward-Backward Selector
            * Permutation Importance
            * RFECV
    * K-Nearest Neighbors
    * Support Vector Regression
    * XGBoost Model
        * One Hot Encoded
        * Target Encoded


* **Model Selection and Analysis**

* **Additional Visualizations**

* **APPENDIX**

# Objective

* Perform exhaustive and accurate data cleaning
* Explore different categorical handling methods
* Build a model that accurately predicts house prices in King County

# Notebook Preparation

In [ ]:
# data processing tools
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from math import sqrt
import itertools

# model tools
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy.stats as stats
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_val_score, RepeatedKFold, train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.inspection import permutation_importance
from sklearn import neighbors
import xgboost as xgb

# Visualization tools
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

# Preprocessing

In [ ]:
# load and look at our king county housing data
df = pd.read_csv('../input/housesalesprediction/kc_house_data.csv')
df

In [ ]:
df.dtypes

## EDA Cleaning

### Scale Time Series

In [ ]:
# converting our date to a proper datetime
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# earliest date
df['date'].min()

In [ ]:
# latest date
df['date'].max()

This data includes a TIME SERIES, and we need to bring our target prices to the same time scale. It's easy to ignore that these homes were sold over the space of a year, but a year is a long time for real estate. King County median prices increased 9.05% from May 2014 to May 2015, and we need to account for this kind of change in our model. We'll do this by appreciating all sale prices into May 2015 using an appreciation rate over the year of 9.05%, and dividing into months (more granularity is possible of course)

In [ ]:
# We're applying a time series multiplier to our rows based on date sold. Our data spans from
# May 2014-May 2015 so we divide it into a 12 month time series.
df.loc[(df['date']>'2014-05-01') & (df['date']<'2014-06-01'), 'time_series'] = 12
df.loc[(df['date']>='2014-06-01') & (df['date']<'2014-07-01'), 'time_series'] = 11
df.loc[(df['date']>='2014-07-01') & (df['date']<'2014-08-01'), 'time_series'] = 10
df.loc[(df['date']>='2014-08-01') & (df['date']<'2014-09-01'), 'time_series'] = 9
df.loc[(df['date']>='2014-09-01') & (df['date']<'2014-10-01'), 'time_series'] = 8
df.loc[(df['date']>='2014-10-01') & (df['date']<'2014-11-01'), 'time_series'] = 7
df.loc[(df['date']>='2014-11-01') & (df['date']<'2014-12-01'), 'time_series'] = 6
df.loc[(df['date']>='2014-12-01') & (df['date']<'2015-01-01'), 'time_series'] = 5
df.loc[(df['date']>='2015-01-01') & (df['date']<'2015-02-01'), 'time_series'] = 4
df.loc[(df['date']>='2015-02-01') & (df['date']<'2015-03-01'), 'time_series'] = 3
df.loc[(df['date']>='2015-03-01') & (df['date']<'2015-04-01'), 'time_series'] = 2
df.loc[(df['date']>='2015-04-01') & (df['date']<'2015-05-01'), 'time_series'] = 1
df.loc[(df['date']>='2015-05-01') & (df['date']<'2015-06-01'), 'time_series'] = 0

# adjusting up our sale prices to account for appreciation using the yearly appreciation for home sales in King County
# from May 2014 to May 2015
monthly_appreciation = .0905/12
df['adj_price'] = round(df['price']*(1+((monthly_appreciation)*df['time_series'])), 0)

# we're renaming our adjusted price column to be price
df.rename(columns={'price': 'orig_price', "adj_price": "price"}, inplace=True)

# renumber our data
df.set_index('price', inplace=True)
df.reset_index(inplace=True)

# we're now going to extract just the month from our datetime, and store that in a new column
# we won't be using our entire date - we're interested in the month that houses are sold
df['month_sold'] = pd.DatetimeIndex(df['date']).month
df.drop('date', axis=1, inplace=True)

### Duplicate Data

In [ ]:
# Look for duplicate data on lat/long

df[df.duplicated(subset=['lat','long'], keep=False)].sort_values('lat')

In [ ]:
# We have a lot of duplicate entries. We're going to keep the later of these entries and hope that if it's an outlier,
# it's caught in our outlier processing later.

df.drop_duplicates(['lat','long'], keep='last', inplace=True)

### Outlier Detection

In [ ]:
# plotting latitude and longitude as a visual scatter plot to look for location outliers

plt.figure(figsize=(25,25))

sns.scatterplot(data=df, x="long", y="lat", hue="price", palette="magma_r");

Using latitude and longitude, we make a visual map of the King County area that lets us see the map outliers. We can see that there are a few sales in locations that are outliers in some way - still in King County but very far away from the metro area on which we are focusing. Sales that are too far outside of our comparison area geographically may add noise to our model. So I'm going to drop the longitude outliers as well. 

This visualization suggests that location is very important to home price. We're going to check that out more directly.

In [ ]:
# drop the properties east of the metropolitan area
df.drop(df[df['long'] > -121.7].index, inplace=True)

# looking for outliers in the percentiles
df.describe()

We see potential outliers in price, sqft_lot, sqft_living, and bedrooms.

In [ ]:
# check how our histograms are looking

df.hist(figsize=(18,15), bins='auto');

We should never remove outliers indiscriminately, especially if they contain real data. Ultimately I opt to remove via IQR on sqft_living and sqft_lot, with reservations. A lot of our variables are not normally shaped, so we can't reliably remove outliers via standard deviation.

For the square footage variables, I ultimately concluded that extremely large houses and lots are so seriously under-represented in the dataset that we won't be able to reliably predict on them anyway and they are better left off.

In order to prevent a lot of data loss in this way, I kept IQR range of 1.6 instead of the standard 1.5

In [ ]:
def iqr_outliers(column):
    """return the lower range and upper range for the data based on IQR*1.6"""
    Q1,Q3 = np.percentile(column , [25,75])
    iqr = Q3 - Q1
    lower_range = Q1 - (1.6 * iqr)
    upper_range = Q3 + (1.6 * iqr)
    return lower_range,upper_range  

# determing our IQR ranges for lot size, sq footage
lotlower,lotupper = iqr_outliers(df.sqft_lot)
sqftlower, sqftupper = iqr_outliers(df.sqft_living)

# dropping the things outside of our lower and upper range
df.drop(df[ (df.sqft_lot > lotupper) | (df.sqft_lot < lotlower) ].index , inplace=True)
df.drop(df[ (df.sqft_living > sqftupper) | (df.sqft_living < sqftlower) ].index , inplace=True)

In [ ]:
# we're using the median house value for a zip code to determine the zip code's sort, so we can visualize the zip code importance

# group our dataframe by zipcode on median home price, sorted ascending. 
zipsorted = pd.DataFrame(df.groupby('zipcode')['price'].median().sort_values(ascending=True))

# rank each zip code and assign rank to new column
zipsorted['rank'] = np.divmod(np.arange(len(zipsorted)), 1)[0]+1

# function that looks up a segment that a data entry belongs to
def make_group(x, frame, column):
    '''Takes in a line, a lookup table, and a target column
    returns value of target column
    ARGUMENTS:
    line from dataframe x
    lookup table frame
    column to return rank'''
    y = frame.loc[(frame.index == x)][column]
    z = np.array(y)
    z[0]
    return z[0]

# make a new column on our dataframe. Look up each zip entry's group, and append to the column.
df['zip_rank'] = df['zipcode'].apply(lambda x: make_group(x, zipsorted, 'rank'))

# apply the median home price per zip code to the data frame
df['median_zip'] = df['zipcode'].apply(lambda x: round(df.loc[df['zipcode']==x]['price'].median(), 0))

In [ ]:
# visualize zip code as a color function, on a plot of price per square footage

fig, ax = plt.subplots(figsize=(20, 15))

ax.scatter(df['sqft_living'], df['price'] /100000, c=df['zip_rank'], cmap='magma_r')

ax.set_xlabel('Square Feet of Living Space', fontsize=12)
ax.set_ylabel('Price in $100,000', fontsize=12)
ax.set_title('Price per Total Square Feet, by Zip Code Median Rank', fontsize=20)
;

This plot of price per total square feet of living space, colored by the zip code median rank, visually shows the importance of location. Let's look at that closer.

In [ ]:
# visualize zip code as a color function

fig, ax = plt.subplots(figsize=(20, 15))

ax.scatter(df['median_zip'], df['price'] /100000, c=df['zip_rank'], cmap='magma_r')

ax.set_xlabel('Median Home Price per Zip', fontsize=12)
ax.set_ylabel('Price in $100,000', fontsize=12)
ax.set_title('Price per Zip Code Median, by Zip Code Median Rank', fontsize=20)
;

Looking at our price per zip code in a different way, we can see a potential issue with our data. Most of our data set is fairly normal and expected. Then we have three zip codes giving us very strong right tails. Then we have ONE zip code way off yonder which may cause problems with our model. 

I'll be eliminating price outliers on a per-zipcode basis using the same IQR range. My goal here is not to strip usable data, but to identify and excise any rogue non-standard sales. If I were to do this dataset-wide, many zipcodes would retain their own outliers. We can see in the visualizations just above that zipcodes each have their own price distribution. So we keep our price outlier detection within zipcode.

In [ ]:
# Eliminating outliers on a per-zipcode basis

zipcodes = df['zipcode'].unique()

for i in zipcodes:
    lower, upper = iqr_outliers(df[df['zipcode'] == i]['price'])
    df.drop(df[ ( (df.price > upper) & (df['zipcode'] == i) ) | ( (df.price < lower)  & (df['zipcode'] == i) ) ].index , inplace=True)


#### Manually locating zip/price outliers

One thing this data set lacks is *type of sale*. We want a model that properly describes the open market, but this dataset definitely includes sales that are outside the market, such as inter-family sales or other sales that don't properly reflect market value. The best way I can think of to find these outliers is to find suspiciously low sales based on location, because we can see in the lat/long scatter that location makes a difference. We will hunt these down manually even though that takes some time.

In [ ]:
# check price stats by zip code and displaying top 30 zip codes by mean
# I'm looking for suspiciously low "min" values

find_zip_outliers = df.groupby('zipcode')['price'].describe()
find_zip_outliers.sort_values('mean', ascending=False).head(35)

# very suspicious values in many zip codes for min
# 98112, 98109, 98033, 98115, 98027, 98116, 98122, 98117, 98136, 98065, 98144, 98072, 98028

In [ ]:
df.loc[(df['price'] <= 250000) & (df['zipcode'] == 98109)]
# There are two listings in this zip code selling for under 250k. We're going to consider this an outlier sale.
# worth noting these are both the exact same property. These are both bad listings.

# It looks like we missed a dupe due to a slight mismatch in lat. Since this is a dupe AND an anomalous price, we are going to dump it.

In [ ]:
df.loc[(df['price'] <= 250000) & (df['zipcode'] == 98033)]
#We're going to consider these outlier sales.

In [ ]:
df.loc[(df['price'] <= 250000) & (df['zipcode'] == 98115)]
#We're going to consider these outlier sales.

In [ ]:
df.loc[(df['price'] <= 250000) & (df['zipcode'] == 98027)]
# listings under 250k in this zip. We'll drop them.

In [ ]:
df.loc[(df['price'] <= 250000) & (df['zipcode'] == 98116)]
# suspicious but in bad condition. We will drop though.

In [ ]:
df.loc[(df['price'] <= 250000) & (df['zipcode'] == 98122)]
# suspicious but VERY tiny. We will leave.

In [ ]:
df.loc[(df['price'] <= 250000) & (df['zipcode'] == 98136)]
# Drop

In [ ]:
df.loc[(df['price'] <= 250000) & (df['zipcode'] == 98065)]
# Drop

In [ ]:
df.loc[(df['price'] <= 250000) & (df['zipcode'] == 98144)]
# Dropping most

In [ ]:
df.loc[(df['price'] <= 250000) & (df['zipcode'] == 98072)]
# Dropping

In [ ]:
df.loc[(df['price'] <= 250000) & (df['zipcode'] == 98028)]
# Dropping

In [ ]:
df.drop(index=[3844, 11436, 1222, 12823, 1975, 3885, 14001, 18897, 3975, 18332, 3987, 881, 11500, 17578, 8133, 9854, 18338, 5573, 5885, 8911, 11298, 2, 1922, 9231, 12306], inplace=True) #16879, 7088,  

Why should we spend the time excising just a few outliers from thousands of rows of data? Because it's the right thing to do to clean well and improve our data quality.

#### Other outlier detection

Now we'll look for bad data in other categories. This is easy to do using describe.

In [ ]:
# looking again at our percentile outliers

df.describe()

In [ ]:
#check what is going on with the sqft_lot15 outliers by sorting descending
df.sort_values('sqft_lot15', ascending=False).head(5)

In [ ]:
# there is something off about these two large and nearly identical entries at the top of the list. We are going to drop these two rows.

df.drop(index=[4611, 9445], inplace=True)

In [ ]:
#check what is going on with the weird bedroom value by sorting descending

df.sort_values('bedrooms', ascending=False).head(5)

In [ ]:
# this value of 33 in 1620 square feet is obviously a mistake. We're going to impute the mean into this field.

# 11 bedrooms in 3000sf, 10 bedrooms in 2920sf, 10 bedrooms in 3610sf are also obviously mistakes
# We're going to impute the mean into the fields for 10 and 11 bedrooms as well.

# we'll also impute the mean into the few bedroom listings with 0 bedrooms

bedroom_mean = round(df['bedrooms'].mean(), 0)
bedroom_mean
df.loc[df['bedrooms'] == 33.0, 'bedrooms'] = bedroom_mean
df.loc[df['bedrooms'] == 11.0, 'bedrooms'] = bedroom_mean
df.loc[df['bedrooms'] == 10.0, 'bedrooms'] = bedroom_mean
df.loc[df['bedrooms'] == 0, 'bedrooms'] = bedroom_mean
df.loc[df['bathrooms'] == 0, 'bathrooms'] = 2

# I remain suspicious of the 9-bedroom entries, but we'll decline from manipulating them further

In [ ]:
# check how our histograms are looking

df.hist(figsize=(18,15), bins='auto');

#Our histograms have improved a lot.

### Missing Data

In [ ]:
# look for missing data
df.isna().sum()

# no missing values

### Binary data

There are a few columns that I might want to be binary flags. I need to see what value choices are in these columns.

In [ ]:
df['yr_renovated'].value_counts(normalize=True).head(10)
# 96% of houses have not been renovated, so we will turn this into a binary flag

In [ ]:
df['waterfront'].value_counts(normalize=True).head(10)
# Hardly any homes on the waterfront, and it's already a binary flag

In [ ]:
df['view'].value_counts(normalize=True).head(10)
# most houses don't have a view, but we have a few distinct choices, so we will KEEP this categorical 

In [ ]:
df['sqft_basement'].value_counts(normalize=True).head(10)
#Many properties don't have basements, but enough do to keep this as a continuous

In [ ]:
#So, will change all values > 0 in those columns to 1
# this will turn renovated into dichotomous choice flags
df.loc[df['yr_renovated'] > 0, 'yr_renovated'] = 1

# now anything that is not a 1 becomes a 0, just in case we missed something weird
df.loc[df['yr_renovated'] != 1, 'yr_renovated'] = 0

# since we're making it a binary flag, we'll rename yr_renovated to renovated
df.rename(columns={'yr_renovated' : 'renovated'}, inplace=True)

In [ ]:
# We're trying out engineering a feature that penalizes or rewards being the smallest or biggest
# property in the neighborhood, as generally you don't want to be either. This is attempting 
# to impose a categorical relationship on how the property relates to its neighbors

df['comparative_sf'] = 0
df.loc[df['sqft_living'] <= (df['sqft_living15']*.75), 'comparative_sf'] = '1'
df.loc[(df['sqft_living'] > (df['sqft_living15']*.75)) & (df['sqft_living'] < (df['sqft_living15']*1.25)), 'comparative_sf'] = '2'
df.loc[df['sqft_living'] >= (df['sqft_living15']*1.25), 'comparative_sf'] = '3'

In [ ]:
# We're making all 0 basement values 1, and 0 view values .01, so that we can log transform this column ( cannot log 0)
df.loc[df['sqft_basement'] == 0, 'sqft_basement'] = 1
df.loc[df['view'] == 0, 'view'] = .01

### Visualize Cleaned Data

In [ ]:
# redo our zip code medians and rankings after data cleaning for visualizations
# this is a repeat of the task we did further up

# apply the median home price per zip code to the data frame again after outlier removal
df['median_zip'] = df['zipcode'].apply(lambda x: round(df.loc[df['zipcode']==x]['price'].median(), 0))

# group our dataframe by zipcode on median home price, sorted ascending. We want to bin like-medians together.
zipsorted = pd.DataFrame(df.groupby('zipcode')['price'].median().sort_values(ascending=True))

# divide our dataframe into groups with entries per group as specified above,
# and assign this number to a new column
zipsorted['rank'] = np.divmod(np.arange(len(zipsorted)), 1)[0]+1

# make a new column on our dataframe. Look up each zip entry's group, and append to the column.
df['zip_rank'] = df['zipcode'].apply(lambda x: make_group(x, zipsorted, 'rank'))

In [ ]:
# re-visualize zip code as a color function, using the median zip after outlier removal. 

fig, ax = plt.subplots(figsize=(20, 15))

ax.scatter(df['median_zip'], df['price'] /100000, c=df['zip_rank'], cmap='magma_r')

ax.set_xlabel('Zip Code by Median Rank', fontsize=12)
ax.set_ylabel('Price in $100,000', fontsize=12)
ax.set_title('Price per Zip Code Median, by Zip Code Median Rank', fontsize=20);

# save visualization to png
#plt.savefig('images/zip_prices.png')

In [ ]:
# visualize zip code as a color function, on a plot of price per square footage

fig, ax = plt.subplots(figsize=(20, 15))

ax.scatter(df['sqft_living'], df['price'] /100000, c=df['zip_rank'], cmap='magma_r')

ax.set_xlabel('Square Feet of Living Space', fontsize=12)
ax.set_ylabel('Price in $100,000', fontsize=12)
ax.set_title('Price per Total Square Footage, by Zip Code Median Rank', fontsize=20)
;

Here's a fun way to see the improvements to our data quality after we clean outliers! A much deeper color map.

In [ ]:
# plotting latitude and longitude as a visual scatter plot. The improved color map actually visually demonstrates
# the removal of extreme price outliers.

plt.figure(figsize=(25,25))

sns.scatterplot(data=df, x="long", y="lat", hue="price", palette="magma_r");

In [ ]:
# We can also use our scatter to see the zip codes by rank! Pretty cool.

plt.figure(figsize=(25,25))

sns.scatterplot(data=df, x="long", y="lat", hue="zip_rank", palette="magma_r");

### Study Target Variable

In [ ]:
#histogram and normal probability plot
sns.distplot(df['price'], fit=norm);
fig = plt.figure()

res = stats.probplot(df['price'], plot=plt)

# our sales price histogram is positively skewed and has a high peak
# Our QQ-plot shows that we have heavy tails with right skew

In [ ]:
#skewness and kurtosis
print("Skewness: %f" % df['price'].skew())
print("Kurtosis: %f" % df['price'].kurt())

# price is highly right skewed
# very positive kurtosis, indicating lots in the tails. We can see those tails in the right skew.

In [ ]:
# log transform our target price to improve normality of distribution
df_target_log = np.log(df['price'])

#histogram and normal probability plot
sns.distplot(df_target_log, fit=norm);
fig = plt.figure()
res = stats.probplot(df_target_log, plot=plt)

# Our target price is more normally distributed when log transformed, so we'll be doing that


## Create Holdout Set

We need to separate our holdout data before any further processing.

The reasons for this are:

    * We will standardize our continuous variables, and you should standardize only on your train set and apply that to your test set.
    * We will be feature engineering on our train set, and applying that later to our test set. We cannot have our test set data leak into our engineered features.

In [ ]:
# set our random seed for the notebook. We could randomize this each time the notebook is run,
# but ultimately we want all of our train/test splits to use the same data
randomstate = 4

# creating our train/validation sets and our test sets
y = pd.DataFrame(df['price'])
train_data, holdout, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=randomstate)

# reset indices to prevent any index mismatches
train_data.reset_index(inplace=True)
train_data.drop('index', axis=1, inplace=True)

holdout.reset_index(inplace=True)
holdout.drop('index', axis=1, inplace=True)

y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

## Feature Engineering

We're going to do some target-encoded feature engineering on our data set, using additive smoothing based on this article by Max Halford https://maxhalford.github.io/blog/target-encoding/

Using this method we'll add features for:
   * smoothed zip code
   * smoothed year
   * smoothed month sold
   * smoothed latitude
   * smoothed longitude
   * lat/long zone as function of smoothed lat and smoothed long

This feature engineering is the primary reason that we chose to separate our holdout set before any other processing. We cannot, under any circumstances, use the test data to inform our train set features!

In [ ]:
# Adding target encoding, which we will opt to try instead of one-hot with a few models

# this function is by Max Halford at the address noted above
def calc_smooth_mean(df, by, on, m, target_df):
    '''input a pandas.DataFrame, a categorical column name, the name of the target column, and a weight .'''
    # Compute the global mean
    mean = df[on].mean() 

    # Compute the number of values and the mean of each group
    agg = df.groupby(by)[on].agg(['count', 'mean'])  
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + m * mean) / (counts + m)

    # Replace each value by the according smoothed mean
    return round(target_df[by].map(smooth), 0) 

# get size of training data
num_of_samples = train_data.shape[0]

# determining minimum number of samples for zip and month to use their
# own mean rather than expanding into the full data set mean 
zip_samples = num_of_samples/train_data['zipcode'].unique().shape[0]
month_samples = num_of_samples/train_data['month_sold'].unique().shape[0]

# create smooth additive encoded variables for zipcode, year built, and monthsold
train_data['zip_smooth'] = calc_smooth_mean(train_data, 'zipcode', 'price', zip_samples, train_data)
train_data['year_smooth'] = calc_smooth_mean(train_data, 'yr_built', 'price', 300, train_data)
train_data['month_smooth'] = calc_smooth_mean(train_data, 'month_sold', 'price', month_samples, train_data)

# Create a wider lat and long zone to calculate an area mean
train_data['lat_zone'] = round(train_data['lat'], 2)
train_data['long_zone'] = round(train_data['long'], 2)

# determing min number samples for lat and long mean
lat_samples = num_of_samples/train_data['lat_zone'].unique().shape[0]
long_samples = num_of_samples/train_data['long_zone'].unique().shape[0]

# calculate smooth mean variables for lat and long, then create an interactive variable describing both together
train_data['lat_smooth'] = calc_smooth_mean(train_data, 'lat_zone', 'price', lat_samples, train_data)
train_data['long_smooth'] = calc_smooth_mean(train_data, 'long_zone', 'price', long_samples, train_data)
train_data['lat_long'] = (np.sqrt(train_data['lat_smooth']) + np.sqrt(train_data['long_smooth']))

## Correlations/Multicollinearity

Time to check and remove any features with high collinearity.

In [ ]:
# look for multicollinearity of features
fig, ax = plt.subplots(figsize=(20, 20))

sns.heatmap(train_data.corr(), center=0, annot=True, fmt=".2f", 
           vmin=-1, vmax=1, cbar_kws={"shrink": .8}, square=True)

# title
plt.title('PEARSON CORRELATION MATRIX', fontsize=18)

plt.show()

In [ ]:
train_data.corr()

In [ ]:
#Get our list of highly correlated feature pairs with following steps:

# save correlation matrix as a new data frame
# converts all values to absolute value
# stacks the row:column pairs into a multindex
# reset the index to set the multindex to seperate columns
# sort values. 0 is the column automatically generated by the stacking
df_correlations = train_data.corr().abs().stack().reset_index().sort_values(0, ascending=False)

# zip the variable name columns in a new column named "pairs"
df_correlations['pairs'] = list(zip(df_correlations.level_0, df_correlations.level_1))

# set index to pairs
df_correlations.set_index(['pairs'], inplace = True)

# rename our results column to correlation
df_correlations.rename(columns={0: "correlation"}, inplace=True)

# Drop 1:1 correlations to get rid of self pairs
df_correlations.drop(df_correlations[df_correlations['correlation'] == 1.000000].index, inplace=True)

# view pairs above 70% correlation and below 90% correlation (engineered features will correlate with each other above 95%)
df_correlations[(df_correlations.correlation>.75) & (df_correlations.correlation<.95)]


median_zip, zip_rank and orig_price are dummy variables that we won't be using, so we can ignore correlations using those for now.

We drop sqft_above and sqft_lot15

In [ ]:
# drop multicollinear features and unneeded features
train_data.drop(['id', 'zip_rank', 'median_zip', 'sqft_above', 'sqft_lot15'], axis=1, inplace=True)

In [ ]:
# Check out our variables correlationg with price
df_correlations = train_data.corr().abs().stack().reset_index().sort_values(0, ascending=False)
df_correlations.loc[df_correlations['level_0'] == 'price'].sort_values(0, ascending=False)

It looks like zip code works better than the lat/long zones I made. We should also note here that zip and lat/long descriptors are all highly correlated, so we should consider if we need to include all of them in our model or just the best one (zipcode)

We can get a sense of the most important features to our price from our correlation table. Zipcode as a plain variable does not correlate, which makes sense, because without some sort of transformation it is an arbitrary unordered number. We can see how transformed as zip_smooth, median_zip, or zip_rank it becomes the MOST important contributor to price. We can see here that big contributors to price include:
    
    * zip code (in some altered form, not as arbitrary number)
    * grade
    * sqft_living
    * bathrooms

## EDA and Process Train Set

### Categoricals

In [ ]:
categoricals = ['floors', 'waterfront', 'renovated', 'comparative_sf', 'month_sold', 'yr_built', 'zipcode', 'lat_long'] #

# make our categorical data frame to work with
df_categoricals = train_data[categoricals]

# binning our year built bins
df_categoricals["year_block"] = pd.qcut(df_categoricals['yr_built'], q=30, labels=np.array(range(1,30+1)))

# binning our latitude bins
df_categoricals["loc_block"] = pd.qcut(df_categoricals['lat_long'], q=50, labels=np.array(range(1,50+1)))

# dropping the original categories we just binned
df_categoricals.drop(['yr_built', 'lat_long'], axis=1, inplace=True)

# telling Pandas that these columns are categoricals
for item in df_categoricals.columns:
    df_categoricals[item] = df_categoricals[item].astype('category')

# temporarily adding price to our dataframe so that we can do some visualizations    
df_categoricals['price'] = train_data['price']

# plot our categoricals as box plots vs price
def boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    x=plt.xticks(rotation=90)

    #visualization categories
categorical=['floors', 'waterfront', 'renovated', 'comparative_sf', 'month_sold', 'year_block', 'zipcode', 'loc_block']
f = pd.melt(df_categoricals, id_vars=['price'], value_vars=categorical)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, size=5)
g = g.map(boxplot, "value", "price")

df_categoricals.drop('price', axis=1, inplace=True)

In [ ]:
# make a processed bins file for use with linear regression
# We're making TWO categorical sets. One is high one hot encoding. One is low one hot encoding, and the 
# categoricals in that one will be target encoded as continuous instead

high_one_hot_cat =  ['floors', 'waterfront', 'renovated','comparative_sf', 'zipcode', 'month_sold', 'year_block', 'loc_block']
low_one_hot_cat =  ['floors', 'waterfront', 'renovated','comparative_sf']

df_cats_high_one_hot = pd.get_dummies(df_categoricals[high_one_hot_cat], prefix=high_one_hot_cat, drop_first=True)
df_cats_low_one_hot = pd.get_dummies(df_categoricals[low_one_hot_cat], prefix=low_one_hot_cat, drop_first=True)

### Continuous

In [ ]:
continuous = ['sqft_living', 'sqft_living15', 'sqft_lot', 'sqft_basement', 'view', 'bedrooms', 'bathrooms', 'condition', 'grade', 'zip_smooth', 'year_smooth', 'month_smooth', 'lat_long'] #'floors',  

# make our continuous frame to work with
x_continuous = train_data[continuous]
x_continuous['price'] = train_data['price']

# plot our smaller choice continuous as box plots vs price
small_cont = ['bedrooms', 'bathrooms', 'condition', 'grade', 'view', 'month_smooth'] 

def boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    x=plt.xticks(rotation=90)

f = pd.melt(x_continuous, id_vars=['price'], value_vars=small_cont)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, size=5)
g = g.map(boxplot, "value", "price")

In [ ]:
# plot our larger continuous as scatter plots vs price
large_cont = ['sqft_living', 'sqft_living15', 'sqft_lot', 'sqft_basement', 'zip_smooth', 'year_smooth', 'month_smooth', 'lat_long']

fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(15,25), sharey=True)

for ax, column in zip(axes.flatten(), large_cont):
    ax.scatter(x_continuous[column], x_continuous['price']/100000, label=column, alpha=.1)
    ax.set_title(f'Sale Price vs {column}')
    ax.set_xlabel(column)
    ax.set_ylabel('Sale Price in $100,000')

fig.tight_layout()

x_continuous.drop('price', axis=1, inplace=True)

    * sqft_living, sqft_living15, zip_smooth, lat_long, sqft_basement(when present) have a very strong visual relationship with price
    * sqft_lot do not appear to have a strong relationship with price
    * year_smooth and month_smooth are unclear, we will investigate further

Fun fact - "grade" was not well defined in the features list. I looked it up and found that grade is "	Classification by construction quality which refers to the types of materials used and the quality of workmanship. Buildings of better quality (higher grade) cost more to build per unit of measure and command higher value." So, it's the quality of building materials, and DEFINITELY worth keeping in our model.

#### Looking for polynomial relationships

In [ ]:
# Checking out our mean sales price for year built scattered versus price shows a polynomial relationship

yearly_prices = train_data.groupby('yr_built')['price'].mean()

plt.scatter(yearly_prices.index, yearly_prices)
plt.title("Linearity check")
plt.xlabel('year built')
plt.ylabel('sales price')
plt.show()

Seems like year built might have a polynomial relationship with price

In [ ]:
# Checking out our mean sales price for month sold scattered versus price shows a polynomial relationship
monthly_prices = train_data.groupby('month_sold')['price'].mean()

plt.scatter(monthly_prices.index, monthly_prices)
plt.title("Linearity check")
plt.xlabel('month sold')
plt.ylabel('sales price')
plt.show()

Month sold may also have a polynomial relationship

In [ ]:
# Checking out our mean sales price for longitude does not show a polynomial relationship
long_prices = train_data.groupby('lat_long')['price'].mean()

plt.scatter(long_prices.index, long_prices)
plt.title("Linearity check")
plt.xlabel('long')
plt.ylabel('sales price')
plt.show()

There does not appear to be a polynomial relationship

In [ ]:
# Checking out our mean sales price for grade shows a polynomial relationship
grade_prices = train_data.groupby('grade')['price'].mean()

plt.scatter(grade_prices.index, grade_prices)
plt.title("Linearity check")
plt.xlabel('long')
plt.ylabel('sales price')
plt.show()

Grade appears to have a polynomial relationship

In [ ]:
# Checking out our mean sales price for grade shows a polynomial relationship
grade_prices = train_data.groupby('sqft_living')['price'].mean()

plt.scatter(grade_prices.index, grade_prices)
plt.title("Linearity check")
plt.xlabel('square footage')
plt.ylabel('sales price')
plt.show()

Square foot living may have a polynomial relationship

#### Finding Interactions

I wrote a function which finds all of the feature combinations possible in our dataset. Then for each combination, the function runs a linear regression with cross validation on 5 folds and gets the r^2 score for the regression including that feature combination. All scores are recorded and r^2 score improvement is assessed, with the resulting table giving the increase in model improvement from a feature combo. 

In [ ]:
def test_feature_combinations(price, variables):
    
    """Function takes in target price and a dataframe of independent variables, and 
    tests model improvement for each combination of variables
    ARGUMENTS:
    Y of target values
    X-dataframe of continuous features
    Returns dataframe of score improvements over base score for each interaction combination"""
    
    # select our estimator and our cross validation plan
    regression = LinearRegression()
    cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=1)
    
    # prepare our scoring dataframe
    scoring_df = pd.DataFrame()
    
    # prepare our lists to store our features and scores as we iterate
    scores = []
    feature1 = []
    feature2 = []
    
    # Get a list of all of our features, and remove our target variable 'price' from the list
    features = list(variables.columns)

    # make a list of all of our possible feature combinations
    feature_combos = itertools.combinations(features, 2)
    feature_combos = list(feature_combos)
    
    # set our y-value as our target variable
    y = price
    
    # prepare our x-value with our independent variables. We do an initial split here in order to run a 
    # linear regression to get a base r^2 on our basic model without interactions
    X = variables
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=randomstate)
    base_score = round(np.mean(cross_val_score(regression, X_train, y_train, scoring='r2', cv=cv)), 4)   
    print("Model base score is ",base_score)
    
    # now we run the regression on each feature combo
    for feature in feature_combos:
        feat1, feat2 = feature[0], feature[1]
        
        # create the test interaction on our data set
        variables['test_interaction'] = variables[feat1] * variables[feat2]
        # create a new X which includes the test interaction and drops our target value
        X = variables
        # make a new split so that our x-splits include the test interaction
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=randomstate)
        
        # Run a linear regression with cross-val just like our base model, and append the score to our scores list
        new_score = round(np.mean(cross_val_score(regression, X_train, y_train, scoring='r2', cv=cv)), 4)
        scores.append(new_score)
        # put feature 1 on a list
        feature1.append(feat1)
        # put feature 2 on a list
        feature2.append(feat2)
        print(feat1, feat2, new_score)
        
        
    
    # load all of our lists into the scoring dataframe
    scoring_df['feature1'] = feature1
    scoring_df['feature2'] = feature2
    scoring_df['scores'] = scores
    scoring_df['improvement'] = scoring_df['scores'] - base_score
    variables.drop('test_interaction', axis=1, inplace=True)
    
    # return our scoring dataframe to the function
    return scoring_df

In [ ]:
# running our function on our continuous variables to look for improvement
# our R2 is much lower for model base score because we aren't including our categorical variables in this improvement assessment

scoring_df = test_feature_combinations(y_train, x_continuous)

In [ ]:
# showing our improvement scores for our interactions

scoring_df.sort_values('improvement', ascending=False)

We see there might be a relationship between square footage and our zip code. This makes sense because the size of the house is differently priced depending on location, as we saw in our visualizations earlier.

In [ ]:
x_continuous['sqft-zip'] = np.sqrt(x_continuous['sqft_living'] * x_continuous['zip_smooth'])

#### Standardize and Transform

In [ ]:
# check out our histograms to see if we should transform our data before scaling

x_continuous.hist(figsize=(18,15), bins='auto');

We will log transform our data, and standardize our continuous inputs.

In [ ]:
# log transform
log_continuous = np.log(x_continuous)

# standardize all of our values with scikit-learn StandardScaler
scaler = StandardScaler()

scaled_continuous = pd.DataFrame(scaler.fit_transform(log_continuous),columns = log_continuous.columns)
scaled_continuous.head(5)

# Our scaled and transformed continuous features

scaled_continuous.hist(figsize=(18,15), bins='auto');

We're ready to make our final continuous data set. At this point we will make TWO continuous sets, because one of the objectives of this notebook is to study *one-hot encoding* versus *mean target encoding*. So we have one data set for our high one-hot encoding features, and one data set for our mean target encoding features. Later in the notebook we will compare these two features sets in models.

In [ ]:
# make a processed bins file for use with linear regression

# our categories for high one-hot encoding and low one-hot encoding
high_one_hot_cont =  ['sqft_living', 'sqft_living15', 'sqft_lot', 'sqft_basement', 'view', 'bedrooms', 'bathrooms', 'grade', 'condition', 'sqft-zip']
low_one_hot_cont =  ['sqft_living', 'sqft_living15', 'sqft_lot', 'sqft_basement', 'view', 'bedrooms', 'bathrooms', 'grade', 'condition', 'zip_smooth', 'year_smooth', 'month_smooth', 'lat_long', 'sqft-zip']  

df_cont_high_one_hot = scaled_continuous[high_one_hot_cont]
df_cont_low_one_hot = scaled_continuous[low_one_hot_cont]

#### Add polynomial features

Now we will add our polynomial features on our low-one-hot set (in the high-one-hot set, these are categoricals)

In [ ]:
def plot_polys(y, xlabel, title):
    '''Takes in a y-axis, x-axis label, and title and plots with various polynomial levels
    ARGUMENTS:
    y axis variable values
    x-axis label
    visualization title'''
    x = y.index
    
    # express numbers as arrays and reshape
    y = np.array(y)
    x = np.array(x)
    x = x.reshape(-1, 1)
    
    # make sure indices match up
    y = y[x[:,0].argsort()]
    x = x[x[:, 0].argsort()]

    # plot figure
    plt.figure(figsize=(16, 8))

    # standard linear regression
    linreg = LinearRegression()
    linreg.fit(x, y)

    # 2nd degree polynomial regression
    poly2 = PolynomialFeatures(degree=2)
    x_poly2 = poly2.fit_transform(x)
    poly_reg2 = LinearRegression()
    poly_reg2.fit(x_poly2, y)

    # third degree polynomial regression 
    poly3 = PolynomialFeatures(degree=3)
    x_poly3 = poly3.fit_transform(x)
    poly_reg3 = LinearRegression()
    poly_reg3.fit(x_poly3, y)

    # predict on x values
    pred = linreg.predict(x)
    pred2 = poly_reg2.predict(x_poly2)
    pred3 = poly_reg3.predict(x_poly3)

    # plot regression lines
    plt.scatter(x, y)
    plt.yscale('log')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel('Average')
    plt.plot(x, pred, c='red', label='Linear regression line')
    plt.plot(x, pred2, c='yellow', label='Polynomial regression line 2')
    plt.plot(x, pred3, c='#a3cfa3', label='Polynomial regression line 3');

In [ ]:
# group by average month sold mean to see relationship
y = train_data.groupby('month_sold')['price'].mean()
plot_polys(y, "Month", "Month Sold Mean")

We'll add month sold as a second degree polynomial.

In [ ]:
# group by average year built mean to see relationship
y = train_data.groupby('yr_built')['price'].mean()
plot_polys(y, "Year Sold", "Year Sold Mean")

We'll add year built as a second degree polynomial.

In [ ]:
# group by grade mean to see relationship
y = train_data.groupby('grade')['price'].mean()
plot_polys(y,'Grade', "Grade Mean")

We will express this as a second degree polynomial.

In [ ]:
# group by grade mean to see relationship
y = train_data.groupby('sqft_living')['price'].mean()
plot_polys(y,'Square Footage', "Square Footage Mean")

This looks better expressed with a third degree polynomial

In [ ]:
# group by grade mean to see relationship
y = train_data.groupby('sqft_lot')['price'].mean()
plot_polys(y,'Lot Square Footage', "Lot Square Footage Mean")

No clear relationship

In [ ]:
# adding our chosen polynomial features

def create_polynomial_array(data, column, num_features):
    values = data[column]
    poly_array = np.array(values)
    poly_array = poly_array.reshape(-1,1)
    poly_fit = PolynomialFeatures(degree=num_features, include_bias=False)
    fit_features = poly_fit.fit_transform(poly_array)
    poly_df = pd.DataFrame(fit_features)
    return poly_df

month_poly = create_polynomial_array(df_cont_low_one_hot, 'month_smooth',2)
year_poly = create_polynomial_array(df_cont_low_one_hot, 'year_smooth', 2)
grade_poly = create_polynomial_array(df_cont_low_one_hot, 'grade', 2)
sq_ft_poly = create_polynomial_array(df_cont_low_one_hot, 'sqft_living', 3)

df_cont_low_one_hot['month1'] = month_poly[1]
df_cont_low_one_hot['grade1'] = grade_poly[1]
df_cont_high_one_hot['grade1'] = grade_poly[1]
df_cont_low_one_hot['year1'] = year_poly[1]
df_cont_low_one_hot['sqft_living2'] = sq_ft_poly[1]
df_cont_high_one_hot['sqft_living2'] = sq_ft_poly[1]
df_cont_low_one_hot['sqft_living3'] = sq_ft_poly[2]
df_cont_high_one_hot['sqft_living3'] = sq_ft_poly[2]

## Process Test Set

We now need to process our holdout in the same way we processed our training set.

In [ ]:
# apply target encoding to test data, using train data to map

# create smooth additive encoded variables for zipcode, year built, and monthsold
holdout['zip_smooth'] = calc_smooth_mean(train_data, 'zipcode', 'price', zip_samples, holdout)
holdout['year_smooth'] = calc_smooth_mean(train_data, 'yr_built', 'price', 300, holdout)
holdout['month_smooth'] = calc_smooth_mean(train_data, 'month_sold', 'price', month_samples, holdout)

# Create a wider lat and long zone to calculate an area mean
holdout['lat_zone'] = round(holdout['lat'], 2)
holdout['long_zone'] = round(holdout['long'], 2)

# calculate smooth mean variables for lat and long, then create an interactive variable describing both together
holdout['lat_smooth'] = calc_smooth_mean(train_data, 'lat_zone', 'price', lat_samples, holdout)
holdout['long_smooth'] = calc_smooth_mean(train_data, 'long_zone', 'price', long_samples, holdout)
holdout['lat_long'] = (np.sqrt(holdout['lat_smooth']) + np.sqrt(holdout['long_smooth']))

### Process Categoricals

In [ ]:
holdout_categoricals = holdout[categoricals]

# binning our year built bins
holdout_categoricals["year_block"] = pd.qcut(holdout_categoricals['yr_built'], q=30, labels=np.array(range(1,30+1)))

# binning our latitude bins
holdout_categoricals["loc_block"] = pd.qcut(holdout_categoricals['lat_long'], q=50, labels=np.array(range(1,50+1)))

holdout_categoricals.drop(['yr_built'], axis=1, inplace=True)

# telling Pandas that these columns are categoricals
for item in holdout_categoricals.columns:
    holdout_categoricals[item] = holdout_categoricals[item].astype('category')

# make a processed bins file for use with linear regression
df_cats_high_one_hot_holdout = pd.get_dummies(holdout_categoricals[high_one_hot_cat], prefix=high_one_hot_cat, drop_first=True)
df_cats_low_one_hot_holdout = pd.get_dummies(holdout_categoricals[low_one_hot_cat], prefix=low_one_hot_cat, drop_first=True)

### Process Continuous

In [ ]:
holdout_continuous = holdout[continuous]

# add feature interaction
holdout_continuous['sqft-zip'] = np.sqrt(holdout_continuous['sqft_living'] * holdout_continuous['zip_smooth'])

# log transform
log_holdout = np.log(holdout_continuous)

# standard scaler
scaled_holdout_continuous = pd.DataFrame(scaler.transform(log_holdout),columns = log_holdout.columns)

# making our two continuous sets
df_cont_high_one_hot_holdout = scaled_holdout_continuous[high_one_hot_cont]
df_cont_low_one_hot_holdout = scaled_holdout_continuous[low_one_hot_cont]

# adding polynomial features

month_poly = create_polynomial_array(df_cont_low_one_hot_holdout, 'month_smooth',2)
year_poly = create_polynomial_array(df_cont_low_one_hot_holdout, 'year_smooth', 2)
grade_poly = create_polynomial_array(df_cont_low_one_hot_holdout, 'grade', 2)
sq_ft_poly = create_polynomial_array(df_cont_low_one_hot_holdout, 'sqft_living', 3)


df_cont_low_one_hot_holdout['month1'] = month_poly[1]
df_cont_low_one_hot_holdout['grade1'] = grade_poly[1]
df_cont_high_one_hot_holdout['grade1'] = grade_poly[1]
df_cont_low_one_hot_holdout['year1'] = year_poly[1]
df_cont_low_one_hot_holdout['sqft_living3'] = sq_ft_poly[2]
df_cont_high_one_hot_holdout['sqft_living3'] = sq_ft_poly[2]
df_cont_low_one_hot_holdout['sqft_living2'] = sq_ft_poly[1]
df_cont_high_one_hot_holdout['sqft_living2'] = sq_ft_poly[1]

## Create Train/Test Final Set

In [ ]:
# make our train sets for one-hot encoded and target-encoded categoricals
X_train_onehot = pd.concat([df_cont_high_one_hot, df_cats_high_one_hot], axis=1)
X_train_encoded = pd.concat([df_cont_low_one_hot, df_cats_low_one_hot], axis=1)

# make our test sets for one-hot encoded and target-encoded categoricals
X_test_onehot = pd.concat([df_cont_high_one_hot_holdout, df_cats_high_one_hot_holdout], axis=1)
X_test_encoded = pd.concat([df_cont_low_one_hot_holdout, df_cats_low_one_hot_holdout], axis=1)

# make our target variable train and test sets, after log transforming our target variable
target = 'price' # target variable
y = np.log(df[target]) # our log-transformed target variable

y_train, y_test = train_test_split(y, test_size=0.2, random_state=randomstate) #

y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)
test_actual = np.exp(y_test)

### Data Sets Reference

Our final data sets include:

* X_train_onehot, X_test_onehot - train/test split predictors for one-hot sets
* X_train_encoded, X_test_encoded - train/test split predictors for encoded sets
* y_train, y_test - target values for all sets
* y - log transformed price
* test_actual - exponentiated y_test prices

# Model Explorations

We're going to evaluate a few different variations of our linear regression model, as well as a few more complex model types. In order to keep track of our results, we'll be making a dictionary to store our model accuracy results.

In [ ]:
# prepare dictionary to store results
models = {}
models['Models'] = []
models['r2'] = []
models['mae'] = []
models['rmse'] = []

Our final data sets include:

* X_train_onehot, X_test_onehot - train/test split predictors for one-hot sets
* X_train_encoded, X_test_encoded - train/test split predictors for encoded sets
* y_train, y_test - target values for all sets
* test_actual - exponentiated y_test prices

## Picking our Base Features

We can use residuals plots to determine if features are important enough to add to our model. If we regress our target on a predictor, and then plot those residuals against a DIFFERENT predictor, our plot will tell us if the new feature might add to our model.

We're going to add features in order of their correlation with price on our correlation heat map, so our base feature is a transformed zip code because it has the strongest correlation.

We regress our target on zip code (we'll use zip_smooth), then we plot our residuals against total square footage.

In [ ]:
x, y = np.array(df['median_zip']).reshape(-1,1), df['price']
z = np.array(df['sqft_living']).reshape(-1,1)

model = LinearRegression()
model.fit(x, y)

test_predictions = model.predict(x)

residuals = y - test_predictions

fig = plt.figure(figsize=(15,10))

# Add labels for x and y axes
plt.xlabel('Total Square Footage')
plt.ylabel('Residuals')

# Add a title for the plot
plt.title('Residuals vs Square Footage - Response is Median_Zip')


plt.scatter(z, residuals, label="sample");

We can see a clear pattern in our residuals, which tells us that adding square footage will add to our model. Now we know this already from our correlation map, but this visual reinforces that information.

We'll do the same regression on zip code again, this time residuals plotted against grade. Again a relationship can be seen.

In [ ]:
x, y = np.array(df['median_zip']).reshape(-1,1), df['price']
z = np.array(df['grade']).reshape(-1,1)

model = LinearRegression()
model.fit(x, y)

test_predictions = model.predict(x)

residuals = y - test_predictions

fig = plt.figure(figsize=(15,10))

# Add labels for x and y axes
plt.xlabel('Grade')
plt.ylabel('Residuals')

# Add a title for the plot
plt.title('Residuals vs Grade - Response is Median_Zip')


plt.scatter(z, residuals, label="sample");

We can see from this graph that Grade has relevance to our model, because the data show a clear pattern. 

Contrast this with yr_built, our lowest correlated feature before we encoded it.

In [ ]:
x, y = np.array(df['median_zip']).reshape(-1,1), df['price']
z = np.array(df['yr_built']).reshape(-1,1)

model = LinearRegression()
model.fit(x, y)

test_predictions = model.predict(x)

residuals = y - test_predictions

fig = plt.figure(figsize=(15,10))

# Add labels for x and y axes
plt.xlabel('Year Built')
plt.ylabel('Residuals')

# Add a title for the plot
plt.title('Residuals vs Year Built - Response is Median_Zip')

plt.scatter(z, residuals, label="sample");

Our scatter looks like a normal residuals plot, implying that the year built does not add much to our model. There's no strong trend exhibited here. If we can see a pattern when we plot residuals vs a different predictor, it can tell us if a feature might add value to our model.

## Linear Regressions

### Basic Model Top Features Only

We're going to build our most baseline model using only the top three features -
    
    * Zip code
    * Grade
    * sqft_living
    * bathrooms

#### One-Hot Encoded Zip Code

In this model we will one-hot encode our 70 zip codes.

In [ ]:
# put together our basic feature set and preprocess

# one-hot encode categorical
base_cat = pd.DataFrame()
base_cat['zipcode'] = df['zipcode']
base_cat['zipcode'] = base_cat['zipcode'].astype('category')
base_cat_processed = pd.get_dummies(base_cat['zipcode'], prefix='zipcode', drop_first=True)
base_cat_processed.reset_index(inplace=True)
base_cat_processed.drop('index', axis=1, inplace=True)

# log transform and standard scale our continuous
base_cont = df[['sqft_living', 'grade', 'bathrooms']]

scaler = StandardScaler()
base_cont_processed = pd.DataFrame(scaler.fit_transform(base_cont),columns = base_cont.columns)

# our log-transformed target variable
y = np.log(df['price'])

#join cat and cont into predictor data frame
#x_base_set = pd.concat([base_cont_processed, base_cat_processed], axis=1)
x_base_set = base_cont_processed.join([base_cat_processed], how='inner') 

# train/test split
x_base_train, x_base_test, y_base_train, y_base_test = train_test_split(x_base_set, y, test_size=0.2, random_state=randomstate)

In [ ]:
# run model for R^2 score

model = LinearRegression()
model.fit(x_base_train, y_base_train)
cv_5 = cross_val_score(model, x_base_train, y_base_train, cv=5)
r2 = cv_5.mean()
r2

Our R-squared of 85.6% indicates that 85.6% of the variance in price can be explained by our features. 

In [ ]:
# apply our model to our test set and get predicted values
test_predictions = model.predict(x_base_test)
test_predictions

# reverse log transform our predicted values
test_predictions_unscaled = np.exp(test_predictions).astype(int)

# get residuals
residuals = test_actual - test_predictions_unscaled

fig = plt.figure(figsize=(20,15))
plt.scatter(test_predictions_unscaled, residuals);
# Residuals plot

In [ ]:
# Calculate our mean absolute error
mae = round(mean_absolute_error(test_actual, test_predictions_unscaled), 2)
rmse = round(np.sqrt(mean_squared_error(test_actual, test_predictions_unscaled)), 2)

# append our results to our lists
models['Models'].append('Bare Bones Features LR - One-Hot Zip')
models['r2'].append(r2)
models['mae'].append(mae)
models['rmse'].append(rmse)

print("MAE:", mae)

Our baseline model has an R^2 of 85.6% which isn't awful, on only a few features. Our MAE is pretty high. We will see if we can improve on that with some other feature selection methods, and even some other model types. But FIRST we will try the same model but using our target encoded zipcode instead.

#### Target Encoded Zip Code

In this model we will use our target encoded  zip codes using zip_smooth.

In [ ]:
# put together our basic feature set and preprocess

x_base_train = X_train_encoded[['sqft_living', 'grade', 'bathrooms', 'zip_smooth']]
x_base_test = X_test_encoded[['sqft_living', 'grade', 'bathrooms', 'zip_smooth']]

In [ ]:
# run model for R^2 score

model = LinearRegression()
model.fit(x_base_train, y_base_train)
cv_5 = cross_val_score(model, x_base_train, y_base_train, cv=5)
r2 = cv_5.mean()
r2

Our R-squared of 78.3% is much lower than when we used our zip code as categoricals. 

In [ ]:
# apply our model to our test set and get predicted values
test_predictions = model.predict(x_base_test)
test_predictions

# reverse log transform our predicted values
test_predictions_unscaled = np.exp(test_predictions).astype(int)

# get residuals
residuals = test_actual - test_predictions_unscaled

fig = plt.figure(figsize=(20,15))
plt.scatter(test_predictions_unscaled, residuals);
# Residuals plot

In [ ]:
# Calculate our mean absolute error
mae = round(mean_absolute_error(test_actual, test_predictions_unscaled), 2)
rmse = round(np.sqrt(mean_squared_error(test_actual, test_predictions_unscaled)), 2)

# append our results to our lists
models['Models'].append('Bare Bones Features LR - Encoded Zip')
models['r2'].append(r2)
models['mae'].append(mae)
models['rmse'].append(rmse)

print("MAE:", mae)

### Linear Regression Model - ALL Features

#### One-hot set

Run a base model with no cross-validation or specific feature selection with ALL possible features. We're going to use our one-hot encoded set which performed better in our first test.

In [ ]:
predictors_train = sm.add_constant(X_train_onehot)
model = sm.OLS(y_train, predictors_train).fit()
model.summary()

There are a good number of features included in this model with a p-value over .05, meaning there is a greater than 5% chance that the results are due to randomness of the sample rather than the feature. A lot of our features have a very low p-value which indicates a very low chance that these results are not affected by the feature. 

Now we perform cross-validation with our base model over 5 splits and get our mean R^2.

In [ ]:
model = LinearRegression()
model.fit(X_train_onehot, y_train)
cv_5 = cross_val_score(model, X_train_onehot, y_train, cv=5)
r2 = cv_5.mean()
r2

Using all of our predictors we get an R^2 of 88.4% which is decent. This indicates that 88.4% of the variance in price can be explained by our features. 

In [ ]:
# apply our model to our test set and get predicted values
test_predictions = model.predict(X_test_onehot)
test_predictions

# reverse log transform our predicted values
test_predictions_unscaled = np.exp(test_predictions)
test_predictions_unscaled = test_predictions_unscaled.flatten().astype(int)

# get residuals
residuals = test_actual - test_predictions_unscaled

fig = plt.figure(figsize=(20,15))
plt.scatter(test_predictions_unscaled, residuals);

# Residuals plot


In [ ]:
# Calculate our mean absolute error
mae = round(mean_absolute_error(test_actual, test_predictions_unscaled), 2)
rmse = round(np.sqrt(mean_squared_error(test_actual, test_predictions_unscaled)), 2)

# append our results to our lists
models['Models'].append('Basic LR - One-Hot')
models['r2'].append(r2)
models['mae'].append(mae)
models['rmse'].append(rmse)

print("MAE:", mae)

##### Study Residuals

In [ ]:
# We need our statsmodels model again to plot residuals
predictors_train = sm.add_constant(X_train_onehot)
model = sm.OLS(y_train, predictors_train).fit()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "sqft_living", fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "sqft_lot", fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "bedrooms", fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "bathrooms", fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "grade", fig=fig)
plt.show()

#### Target Encoded Set

Run a base model with no cross-validation or specific feature selection with ALL possible features. We're using our target categorical encoded set which performed worse in our first test.

In [ ]:
predictors_train = sm.add_constant(X_train_encoded)
model = sm.OLS(y_train, predictors_train).fit()
model.summary()

There are a good number of features included in this model with a p-value over .05, meaning there is a greater than 5% chance that the results are due to randomness of the sample rather than the feature. A lot of our features have a very low p-value which indicates a very low chance that these results are not affected by the feature. 

Now we perform cross-validation with our base model over 5 splits and get our mean R^2.

In [ ]:
model = LinearRegression()
model.fit(X_train_encoded, y_train)
cv_5 = cross_val_score(model, X_train_encoded, y_train, cv=5)
r2 = cv_5.mean()
r2

Using all of our predictors we get an R^2 of 82.3% which is less than when we use our zip code and month as categoricals. So in our linear models, we will use our one-hot encoded data set.

In [ ]:
# apply our model to our test set and get predicted values
test_predictions = model.predict(X_test_encoded)
test_predictions

# reverse log transform our predicted values
test_predictions_unscaled = np.exp(test_predictions)
test_predictions_unscaled = test_predictions_unscaled.flatten().astype(int)

# get residuals
residuals = test_actual - test_predictions_unscaled

fig = plt.figure(figsize=(20,15))
plt.scatter(test_predictions_unscaled, residuals);

# Residuals plot


In [ ]:
# Calculate our mean absolute error
mae = round(mean_absolute_error(test_actual, test_predictions_unscaled), 2)
rmse = round(np.sqrt(mean_squared_error(test_actual, test_predictions_unscaled)), 2)

# append our results to our lists
models['Models'].append('Basic LR - Encoded')
models['r2'].append(r2)
models['mae'].append(mae)
models['rmse'].append(rmse)

print("MAE:", mae)

### Linear Regression - Feature Selectors

Feature selectors are different methods to help us pick which features we want to use in our model. In our example above where we used ALL predictors in our linear regression, several of our features had a p-value over .05, which indicates that there is more than a 5% chance that the changes attributed to that feature were actually by random chance. We want features where our p-value is below a threshold that we specify where we are reasonably confident that the feature is contributing to the model and not by random chance.

We're going to use our one-hot encoded set for this because it performs better than our target encoding set.

#### Forward-Backward Selector

First we'll try a simple forward-backward feature selection model based on p-value, using a statsmodel OLS linear regression model. This selector starts with zero features, internally runs the model for each feature individually, and adds the lowest p-value feature to its list to include. It then runs the model again with the original feature included and tries adding each other feature individually. It will either add the next best feature under the threshold or remove an existing feature if it is no longer within the threshold. This process iterates until all features in the model are under the p-value threshold.

This selector was written by David Dale: https://datascience.stackexchange.com/questions/937/does-scikit-learn-have-forward-selection-stepwise-regression-algorithm

In [ ]:
'''def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ 
    Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = included[pvalues.argmax()]
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included


result = stepwise_selection(X_train_onehot, y_train, verbose=True)

print('resulting features:', result)
'''

result = ['sqft_living15', 'grade', 'sqft-zip', 'sqft_living2', 'view', 'zipcode_98117', 'zipcode_98103', 'zipcode_98115', 'zipcode_98107', 'zipcode_98075', 'condition', 'zipcode_98002', 'zipcode_98032', 'zipcode_98092', 'zipcode_98023', 'zipcode_98022', 'zipcode_98039', 'zipcode_98038', 'zipcode_98109', 'zipcode_98119', 'zipcode_98122', 'zipcode_98102', 'zipcode_98116', 'zipcode_98105', 'zipcode_98136', 'renovated_1', 'zipcode_98125', 'zipcode_98126', 'zipcode_98199', 'zipcode_98034', 'zipcode_98144', 'zipcode_98008', 'zipcode_98033', 'zipcode_98053', 'zipcode_98133', 'zipcode_98118', 'zipcode_98112', 'sqft_basement', 'zipcode_98005', 'zipcode_98004', 'zipcode_98007', 'zipcode_98052', 'zipcode_98177', 'zipcode_98029', 'zipcode_98027', 'zipcode_98155', 'floors_3.0', 'zipcode_98040', 'zipcode_98006', 'zipcode_98074', 'zipcode_98072', 'zipcode_98011', 'zipcode_98028', 'sqft_living', 'year_block_30', 'waterfront_1', 'zipcode_98065', 'zipcode_98077', 'zipcode_98106', 'zipcode_98108', 'zipcode_98056', 'zipcode_98059', 'zipcode_98045', 'zipcode_98166', 'zipcode_98019', 'zipcode_98146', 'zipcode_98014', 'zipcode_98024', 'month_sold_6', 'month_sold_5', 'year_block_29', 'sqft_lot', 'bathrooms', 'month_sold_4', 'year_block_3', 'year_block_5', 'zipcode_98058', 'zipcode_98178', 'zipcode_98055', 'zipcode_98070', 'year_block_4', 'floors_2.0', 'loc_block_46', 'month_sold_7', 'month_sold_3', 'year_block_18', 'year_block_13', 'loc_block_47', 'month_sold_8', 'loc_block_37', 'loc_block_33', 'zipcode_98148', 'floors_2.5', 'year_block_11', 'loc_block_31', 'loc_block_36', 'loc_block_34', 'loc_block_2', 'loc_block_40', 'loc_block_45', 'loc_block_50', 'loc_block_49', 'loc_block_41', 'loc_block_48', 'loc_block_38', 'loc_block_44', 'loc_block_39', 'loc_block_42', 'loc_block_35', 'loc_block_43', 'loc_block_29', 'loc_block_28', 'loc_block_30', 'loc_block_27', 'loc_block_32', 'loc_block_25', 'year_block_15', 'loc_block_26', 'year_block_12', 'year_block_14', 'year_block_16', 'year_block_17', 'year_block_10', 'loc_block_10', 'year_block_9', 'zipcode_98198', 'year_block_27', 'loc_block_3', 'month_sold_9', 'grade1', 'zipcode_98168']

In [ ]:
# Run our linear regression again, using only the features recommended by our feature selector

X_train_refined = X_train_onehot[result]
X_test_refined = X_test_onehot[result]

predictors_int = sm.add_constant(X_train_refined)
model = sm.OLS(y_train, predictors_int).fit()
model.summary()

Same score, but no features above our threshold.

In [ ]:
print("{} predictors used".format(len(result)))

In [ ]:
model = LinearRegression()
model.fit(X_train_refined, y_train)
cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=1)

cv_5 = cross_val_score(model, X_train_refined, y_train, cv=cv)
r2 = cv_5.mean()
r2

In [ ]:
# apply our model to our test set and get predicted values
test_predictions_refined = model.predict(X_test_refined)

# reverse log transform our predicted values
test_predictions_refined_unscaled = np.exp(test_predictions_refined)
test_predictions_refined_unscaled=test_predictions_refined_unscaled.flatten()

# get residuals
residuals = test_actual - test_predictions_refined_unscaled

# plot residuals
fig = plt.figure(figsize=(20,15))
plt.scatter(test_predictions_refined_unscaled, residuals)

In [ ]:
# get mean absolute error
mae = round(mean_absolute_error(test_actual, test_predictions_refined_unscaled), 2)
rmse = round(np.sqrt(mean_squared_error(test_actual, test_predictions_refined_unscaled)), 2)

# append our results to our lists
models['Models'].append('Forw-Back Selector')
models['r2'].append(r2)
models['mae'].append(mae)
models['rmse'].append(rmse)

print("MAE:", mae)

#### Permutation Importance

Next we're going to select features using a method called permutation importance. This is a great model-agnostic method that you can use with any model type, and the way it works is very easy to understand. After fitting the model, it calculates a baseline R^2 score. Then for each feature, it scrambles the inputs of that feature, turnings its contribution into noise. The model is evaluated again with the feature scrambled, and the change in overall R^2 is logged as the importance for that feature. After scrambling all features, each feature has been assigned an importance based on the R^2 reduction. You can then select the features that had an effect on R^2 based on your own threshold (I kept anything >= .001) and throw out the remaining features.

You can learn more about this underrated feature selection method here: https://explained.ai/rf-importance/
The article focuses on Random Forest, but discusses permutation importance as an excellent feature selection method for any model type.

In [ ]:
'''model = LinearRegression()
model.fit(X_train_onehot, y_train)


r = permutation_importance(model, X_train_onehot, y_train,
                           n_repeats=15,
                            random_state=0,
                          n_jobs=-1)

importances = {}

for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] >= 0.001:
        importances[X_train_onehot.columns[i]] = r.importances_mean[i]
    else: continue
        
importances

important_features = list(importances.keys())
important_features  '''

In [ ]:
important_features = ['sqft_living','zipcode_98103','zipcode_98115','zipcode_98117','zipcode_98004',
                      'zipcode_98112','zipcode_98033','zipcode_98199','zipcode_98105','grade','zipcode_98052',
                      'zipcode_98122','zipcode_98107','zipcode_98034','zipcode_98125','zipcode_98040','zipcode_98116',
                      'zipcode_98133','zipcode_98006','zipcode_98119','zipcode_98136','sqft-zip','zipcode_98144',
                      'zipcode_98155','zipcode_98126','zipcode_98118','zipcode_98109','zipcode_98027','zipcode_98008',
                      'zipcode_98177','zipcode_98102','zipcode_98029','zipcode_98053','zipcode_98074','zipcode_98005',
                      'zipcode_98056','zipcode_98059','sqft_living2','zipcode_98028','zipcode_98075','zipcode_98039',
                      'zipcode_98072','zipcode_98007','zipcode_98065','sqft_living15','zipcode_98011','loc_block_46',
                      'loc_block_47','loc_block_50','zipcode_98106','loc_block_49','condition','loc_block_45',
                      'loc_block_48','loc_block_41','loc_block_36','sqft_basement','zipcode_98146','zipcode_98166',
                      'loc_block_38','loc_block_44','loc_block_33','loc_block_37','loc_block_40','loc_block_42',
                      'loc_block_39','view','loc_block_31','loc_block_43','loc_block_34','zipcode_98045','zipcode_98108',
                      'loc_block_35','zipcode_98019','zipcode_98058','loc_block_29','zipcode_98038','floors_2.0',
                      'floors_3.0','zipcode_98077','loc_block_30','loc_block_32','loc_block_27','loc_block_28',
                      'zipcode_98178','bathrooms','month_sold_6','month_sold_5','zipcode_98055','sqft_lot','month_sold_4',
                      'loc_block_25','year_block_18','year_block_13','loc_block_26','month_sold_7','loc_block_22',
                      'loc_block_24','month_sold_3','loc_block_10','renovated_1','year_block_11','loc_block_18',
                      'year_block_30','loc_block_23','year_block_14','waterfront_1','year_block_12','loc_block_21',
                      'year_block_16','loc_block_16','zipcode_98168','loc_block_20']

In [ ]:
# set the features recommended by our feature selector

X_train_perm = X_train_onehot[important_features]
X_test_perm = X_test_onehot[important_features]

predictors_int = sm.add_constant(X_train_perm)
model = sm.OLS(y_train, predictors_int).fit()
model.summary()

In [ ]:
print("{} predictors used".format(len(important_features)))

In [ ]:
model = LinearRegression()
model.fit(X_train_perm, y_train)
cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=1)

cv_5 = cross_val_score(model, X_train_perm, y_train, cv=cv)
r2 = cv_5.mean()
r2

In [ ]:
# apply our model to our test set and get predicted values
test_predictions_refined = model.predict(X_test_perm)

# reverse log transform our predicted values
test_predictions_refined_unscaled = np.exp(test_predictions_refined)
test_predictions_refined_unscaled=test_predictions_refined_unscaled.flatten()

# get residuals
residuals = test_actual - test_predictions_refined_unscaled

# plot residuals
fig = plt.figure(figsize=(20,15))
plt.scatter(test_predictions_refined_unscaled, residuals)

In [ ]:
# get mean absolute error
mae = round(mean_absolute_error(test_actual, test_predictions_refined_unscaled), 2)
rmse = round(np.sqrt(mean_squared_error(test_actual, test_predictions_refined_unscaled)), 2)

# append our results to our lists
models['Models'].append('Permutation Importance Selector')
models['r2'].append(r2)
models['mae'].append(mae)
models['rmse'].append(rmse)

print("MAE:", mae)

#### Recursive Feature Elimination with Cross Validation - Linear Regression

RFECV is a reverse forward-backward selector. It starts the model with all features in use then removes the weakest one, and iterates until the best feature set is found. It uses integrated cross-validation to determine the optimal set of features in the model with the best cross-validated score. We score on mean absolute error.

In [ ]:
# Using sklearn RFECV to perform integrated CV while picking the number of features
# picks the number of features itself

model = LinearRegression()
#cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=1)

rfecv = RFECV(estimator=model, step=1, cv=5, scoring='neg_mean_absolute_error')

# fit model to train set
rfecv.fit(X_train_onehot, y_train)

# print optimal number of features
print('Optimal number of features: {}'.format(rfecv.n_features_))

In [ ]:
dset = pd.DataFrame()
dset['attr'] = X_train_onehot.columns
dset['used'] = rfecv.support_

# make a list of the features used in the rfecv
rfecv_result = list(dset[(dset['used'] == True)]['attr'])

# Show the features that RFECV did not use
dset[dset['used']==False]

In [ ]:
# Run our linear regression again in statsmodels, using the features recommended by our feature selector

X_train_rfecv = X_train_onehot[rfecv_result]
X_test_rfecv = X_test_onehot[rfecv_result]

predictors_int = sm.add_constant(X_train_rfecv)
model = sm.OLS(y_train, predictors_int).fit()
model.summary()

In [ ]:
# getting the r2 score of our best feature set
r2 = model.rsquared
r2

RFECV opted to include all the features, which I find interesting.


In [ ]:
plt.figure(figsize=(16, 9))
plt.title('Recursive Feature Elimination with Cross-Validation', fontsize=18, fontweight='bold', pad=20)
plt.xlabel('Number of features selected', fontsize=14, labelpad=20)
plt.ylabel('R2', fontsize=14, labelpad=20)
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_, color='#303F9F', linewidth=3)

plt.show()

In [ ]:
# predict on new data
rfecv_predictions = rfecv.predict(X_test_onehot)

rfecv_predictions_unscaled = np.exp(rfecv_predictions)
rfecv_predictions_unscaled = rfecv_predictions_unscaled.flatten().astype(int)

# get residuals
residuals = test_actual - rfecv_predictions_unscaled

#plot residuals 
fig = plt.figure(figsize=(20,15))
plt.scatter(rfecv_predictions_unscaled, residuals);

In [ ]:
# get mean absolute error
mae = round(mean_absolute_error(test_actual, rfecv_predictions_unscaled), 2)
rmse = round(np.sqrt(mean_squared_error(test_actual, rfecv_predictions_unscaled)), 2)

models['Models'].append('RFECV')
models['r2'].append(r2)
models['mae'].append(mae)
models['rmse'].append(rmse)

print("MAE:", mae)

## K-Nearest Neighbors Model

K-Nearest Neighbors is more commonly used for classification. Its basic premise is to determine "what is this like" in making a prediction, by looking at other things that are close in value/type. We can pick how many neighbors it assesses to make a classification. As we will see, it doesn't work very well for this type of application (or, I've not tuned the hyperparameters properly and/or don't know how to use it well).

We're using our target encoded data set on this.

In [ ]:
mae_val = [] #to store mae values for different k

# checks mean absolute error scores on k from 1 to 25
for K in range(4, 20):
    K = K+1
    
    # set up the KNN regressor
    model = neighbors.KNeighborsRegressor(n_neighbors = K)

    model.fit(X_train_encoded, y_train)  #fit the model
    pred=model.predict(X_test_encoded) #make prediction on test set
    error = mean_absolute_error(y_test,pred) #calculate mae
    mae_val.append(error) #store mae values
    print('MAE value for k= ' , K , 'is:', error)
    
# gets optimal k-value based on score minimum
index_min = np.argmin(mae_val) + 1

# makes model and fits using optimal k
model = neighbors.KNeighborsRegressor(n_neighbors = index_min)
model.fit(X_train_encoded, y_train)  #fit the model

# Get R^2 with cv
scores = cross_val_score(model, X_train_encoded, y_train, scoring='r2', cv=5, n_jobs=-1, error_score='raise')
r2 = np.mean(scores)
r2

In [ ]:
#make prediction on test set
pred_knn = model.predict(X_test_encoded)
pred_knn = np.exp(pred_knn).flatten().astype(int)

# get residuals
residuals = test_actual - pred_knn

# plot residuals
fig = plt.figure(figsize=(20,15))
plt.scatter(pred_knn, residuals)

In [ ]:
# get mean absolute error

mae = round(mean_absolute_error(test_actual, pred_knn), 2)
rmse = round(np.sqrt(mean_squared_error(test_actual, pred_knn)), 2)

models['Models'].append('KNN')
models['r2'].append(r2)
models['mae'].append(mae)
models['rmse'].append(rmse)

print("MAE:", mae)

## Support Vector Regression

Support vector regression is a form of regression that allows us to define the acceptable error in our model and then finds the line that best fits the data, according to our specifications. This is really useful with something like housing price predictions, where we are ok with our prediction being within a certain dollar amount. SVR will attempt to get all of the predictions within that dollar amount when possible. This will result in a fit line that is different than a linear regression would have produced, but should result in a lower absolute error, which is a reasonable scoring metric for housing price predictions.

We're going to use sklearn's GridSearchCV to find the optimal hyperparameters to use with our SVM! Here are the parameters we are trying out:

* kernel: linear is parametric, and rbf is non-parametric. One of these should perform better. Our data is not totally normal, so it might be rbf.
* epsilon: This value is how much error we're ok with accepting without assigning a penalty to the model
* C: The error that we will accept from a point outside our epsilon

Our C and epsilon need to be in scale with our output variable, which is our log-transformed price.


We COULD tune all of these at once with one huge grid search. I try to avoid this because it takes several hours, and we can get a similar result with careful iterative tuning.

The tuning iterations are found in the APPENDIX.

In [ ]:
# running our model with the optimal parameters

svr = SVR(kernel = 'rbf', gamma = 'auto', C = 10, epsilon = .01)

# fitting our estimator to train data
svr.fit(X_train_refined, y_train)

# getting R^2 with cv
cv_5 = cross_val_score(svr, X_train_refined, y_train, cv=5, n_jobs=-4)
r2 = cv_5.mean()
r2

In [ ]:
# make new predictions on test
predictions_SVR = svr.predict(X_test_refined)
predictions_SVR_unscaled = np.exp(predictions_SVR)

# get residuals
residuals = test_actual - predictions_SVR_unscaled

# plot residuals
fig = plt.figure(figsize=(20,15))
plt.scatter(predictions_SVR_unscaled, residuals)

In [ ]:
# get mean absolute error

mae = round(mean_absolute_error(test_actual, predictions_SVR_unscaled), 2)
rmse = round(np.sqrt(mean_squared_error(test_actual, predictions_SVR_unscaled)), 2)

models['Models'].append('SVR')
models['r2'].append(r2)
models['mae'].append(mae)
models['rmse'].append(rmse)

print("MAE:", mae)

## XGBoost

We're now going to work with a different model type that is entirely different from linear regression. 

There's conflicting information on whether we should use one-hot encoding, or target encoding. We'll solve this by trying both and figuring out what works best for our data set.

### One-Hot Encoding

We need to slightly redo our one-hot encodings to not drop the first entry.

In [ ]:
# categoricals with first not dropped for tree
boost_train_cats = pd.get_dummies(df_categoricals[high_one_hot_cat], prefix=high_one_hot_cat, drop_first=False)
boost_test_cats= pd.get_dummies(holdout_categoricals[high_one_hot_cat], prefix=high_one_hot_cat, drop_first=False)

# continuous not transformed or standardized
boost_train_continuous = x_continuous[high_one_hot_cont]
boost_holdout_continuous = holdout_continuous[high_one_hot_cont]

# decision tree regression train and test sets
x_train_boost = pd.concat([boost_train_continuous, boost_train_cats], axis=1)
x_test_boost = pd.concat([boost_holdout_continuous, boost_test_cats], axis=1)

# redoing our y_train and y_test as non-log transformed
y = df[target] # our target variable

# creating our train/validation sets and our test sets
y_train, y_test = train_test_split(y, test_size=0.2, random_state=randomstate)

# reset indices to avoid index mismatches
y_train = pd.DataFrame(y_train)
y_train.reset_index(inplace=True)
y_train.drop('index', axis=1, inplace=True)

y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True)
y_test.drop('index', axis=1, inplace=True)

Gradient boosting uses decision trees to learn about outcomes, with trees being added to the model one at a time and existing trees in the model are not changed. Each successive tree tries to improve upon the predictions of the first one, with the weights of the various decision points being updated each time. Gradient boosting uses the residuals to improve its next tree prediction. Overall much more opaque of a process than linear regression.

Gradient Boosting performs best with optimal parameter tuning. We're going to use sklearn's GridSearchCV to find the optimal hyperparameters to use with our gradient booster! Here are the parameters we are trying out:

* n_estimators: Number of boosts to perform. Gradient boosting is pretty robust to over-fitting so more is usually better
* max_depth: This determines how many tree nodes the estimator looks at before making a prediction. We don't know what is best here, so we are trying things from 2-4 to see what works the best
* min_child_weight: Min sum of instance weight needed in a child
* gamma: Minimum loss reduction to make another partition on a leaf node. Higher results in more conservative algorithm.
* subsample: Ratio of training sets. .5 means that it will sample half the training data before making trees. Occurs with each boosting iteration.
* colsample_by_tree: ratio of columns when making a tree
* alpha: L1 regularization. Higher will make model more conservative.
* learning_rate: Tuning this setting alters how much the model corrects after it runs a boost. .1 is a common rate and we will test a lower and higher rate as well.

XGBoost parameter tuning is located in the APPENDIX.

In [ ]:
best_xgb_model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 5,
                min_child_weight = 8,
                gamma = .1,
                colsample_bytree=.8,
                reg_alpha = 75,
                reg_lambda=3,
                seed=42,
                missing=0,
                eval_metric='mae' )
best_xgb_model.fit(x_train_boost, y_train)

cv_5 = cross_val_score(best_xgb_model, x_train_boost, y_train, cv=5)
r2 = cv_5.mean()
r2

In [ ]:
# make prediction
preds = best_xgb_model.predict(x_test_boost)

#log residuals
residuals = test_actual - preds

# plot residuals
fig = plt.figure(figsize=(20,15))
plt.scatter(preds, residuals)

In [ ]:
# Calculate our mean absolute error

mae = round(mean_absolute_error(test_actual, preds), 2)
rmse = round(np.sqrt(mean_squared_error(test_actual, preds)), 2)

models['Models'].append('XGBoost - One Hot Encoding')
models['r2'].append(r2)
models['mae'].append(mae)
models['rmse'].append(rmse)

print("MAE:", mae)

### Target Encoding

We're now going to run the same XGBoost regressor, but using our data set with the target encoding. Rather than try to figure out which one is better, we will just try both!

In [ ]:
# categoricals with first not dropped for tree
boost_train_cats = pd.get_dummies(df_categoricals[low_one_hot_cat], prefix=low_one_hot_cat, drop_first=False)
boost_test_cats= pd.get_dummies(holdout_categoricals[low_one_hot_cat], prefix=low_one_hot_cat, drop_first=False)

# continuous not transformed or standardized
boost_train_continuous = x_continuous[low_one_hot_cont]
boost_holdout_continuous = holdout_continuous[low_one_hot_cont]

# decision tree regression train and test sets
x_train_boost_encoded = pd.concat([boost_train_continuous, boost_train_cats], axis=1)
x_test_boost_encoded = pd.concat([boost_holdout_continuous, boost_test_cats], axis=1)

# redoing our y_train and y_test as non-log transformed
y = df[target] # our target variable

# creating our train/validation sets and our test sets
y_train, y_test = train_test_split(y, test_size=0.2, random_state=randomstate)

# reset indices to avoid index mismatches
y_train = pd.DataFrame(y_train)
y_train.reset_index(inplace=True)
y_train.drop('index', axis=1, inplace=True)

y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True)
y_test.drop('index', axis=1, inplace=True)

This model is entirely different than linear regression. Gradient boosting uses decision trees to learn about outcomes, with trees being added to the model one at a time and existing trees in the model are not changed. Each successive tree tries to improve upon the predictions of the first one, with the weights of the various decision points being updated each time. Gradient boosting uses the residuals to improve its next tree prediction. Overall much more opaque of a process than linear regression.

Gradient Boosting performs best with optimal parameter tuning. We're going to use sklearn's GridSearchCV to find the optimal hyperparameters to use with our gradient booster! Here are the parameters we are trying out:

* n_estimators: Number of boosts to perform. Gradient boosting is pretty robust to over-fitting so more is usually better
* max_depth: This determines how many tree nodes the estimator looks at before making a prediction. We don't know what is best here, so we are trying things from 2-4 to see what works the best
* min_child_weight: 
* gamma: 
* subsample:
* colsample_by_tree: 
* reg_alpha: 
* learning_rate: Tuning this setting alters how much the model corrects after it runs a boost. .1 is a common rate and we will test a lower and higher rate as well.

XGBoost parameter tuning is located in the appendix.

In [ ]:
model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 3,
                min_child_weight=20,
                gamma = .1,
                colsample_bytree=.9,
                reg_alpha=.15,
                reg_lambda=3,
                seed=42,
                missing=0,
                eval_metric='mae' )
best_xgb_model.fit(x_train_boost_encoded, y_train)


cv_5 = cross_val_score(best_xgb_model, x_train_boost_encoded, y_train, cv=5)
r2 = cv_5.mean()
r2

In [ ]:
# make prediction
preds = best_xgb_model.predict(x_test_boost_encoded)

#log residuals
residuals = test_actual - preds

# plot residuals
fig = plt.figure(figsize=(20,15))
plt.scatter(preds, residuals)

In [ ]:
# Calculate our mean absolute error

mae = round(mean_absolute_error(test_actual, preds), 2)
rmse = round(np.sqrt(mean_squared_error(test_actual, preds)), 2)

models['Models'].append('XGBoost - Encoded')
models['r2'].append(r2)
models['mae'].append(mae)
models['rmse'].append(rmse)

print("MAE:", mae)


# Model Selection

We ran several different types of models, and logged the r^squared and mean absolute error for each model type. Which model performed the best for us?

In [ ]:
# make data frame from our models dictionary
model_types = pd.DataFrame(models)

# sort data frame by mae and reset index
model_types = model_types.sort_values('mae', ascending=True).reset_index()
model_types.drop('index',axis=1, inplace=True)
model_types.set_index('Models', inplace=True)

model_types

In [ ]:
# plot model mae

plt.figure(figsize=(15,10))
plt.plot(model_types['mae'])
plt.title("Mean Average Error")
plt.xticks(rotation=90)
plt.xlabel('Model')
plt.ylabel("MAE");

Ultimately we will select the sklearn SVM method for our model. 

## Analysis

> Our final model utilizes a combination of continuous variables and one-hot-encoded categoricals to produce a support vector machine regression with R^2 of 89.3%, a mean absolute error of 52k, and a root mean squared error of 78k. I tried several different zip code transformations including polynomial features, mean target encoding, lower-granularity binning, and median rank as a continuous, and ALL of these efforts resulted in a lower R^2 and higher mean absolute error, leading to a final decision to one-hot encode all 70 zip codes individually. Similar efforts on other categoricals such as age and month sold also did not improve the model over one-hot encoding. This resulted in the greatest accuracy despite a model that is more "messy" with a large number of features.

### What are the primary factors influencing housing prices in the King County metro area?
> As square footage increases so does quality of materials. Most importantly you can see the upward price trend with both increased square footage and materials grade. I was intrigued that our lower bound of data points is very linear, but as our square footage increases, the upper bound gradually breaks away with higher variance. 

>I ranked the 70 zip codes in King County by median home value, and used those ranks to color our data points.  Our low median zip codes have a low price per square footage, and price per square foot increases with zip code median, which makes sense, but also shows the importance of zip code to pricing. I found it interesting that while most zip codes exhibit a clear trend of price per square foot decreasing with increased total square footage, which is entirely normal, certain very high value zip codes seem to retain their high price per square foot regardless of total square footage. Certain zip codes seem immune to the usual price per square foot decay. 

> As they say, location is everything, and it is the primary influencing factor for a home price in the King County metro area. Our darkest areas, and therefore highest value sales, are clustered in and around Seattle to the west of Lake Washington and into the eastern lake cities of Bellevue and Redmond which are the technical employer hubs of the region. As we move away from Seattle and the tech hubs into the suburbs, our prices clearly go down.

> These three features alone explain 85% of the price variance.

### Can we effectively use a regression model based system for realtors to determine a proper list price?
> Our model, while explaining over 89.3% of the price variance with our features, was nonetheless far from accurate in absolute terms. A mean average error of 52k in either direction is a huge variance to a home price - one that is so large that it renders the prediction much less meaningful and useful. Other models need to be explored, better data needs to be sourced, or easy-to-use features that an average realtor is capable of evaluating/acquiring should be added to the model to improve its predictive quality. The model is providing at best a baseline starting point.


# Additional Visualizations

## Feature Visualizations

In [ ]:
# get the columns we are going to make visualizations from
viz_df = df[['price', 'sqft_living', 'median_zip', 'zip_rank', 'grade']]
viz_df['pr_sf'] = round(viz_df['price']/viz_df['sqft_living'], 2)
viz_df

In [ ]:
# make simpler variables for our visualiation variables
viz_target = viz_df['price']/100000
viz_sqft = viz_df['sqft_living']
viz_grade = viz_df['grade']
viz_zip = viz_df['zip_rank'] 
viz_zip2 = viz_df['median_zip']
viz_pr_sf = viz_df['pr_sf']

In [ ]:
fig, ax=plt.subplots(figsize=(20,15)) # prepare our figure

sns.set(font_scale = 1.5) # set our font scale bigger for this vis

# scatter our data
scatter2 = sns.scatterplot(x="sqft_living", y=viz_target, data=viz_df, hue='grade', palette='magma_r')

# label axes and title
ax.set_xlabel('Total Square Footage', fontsize=16)
ax.set_ylabel('Price in $100,000', fontsize=16)
ax.set_title("Price to Total Square Footage\nby Grade of Materials", fontsize=20)

# label and position our legend
plt.legend(title='Grade', loc='upper left', title_fontsize=20);

# save visualization to png
#plt.savefig('images/pr_grade.png')

In [ ]:
fig, ax = plt.subplots(figsize=(20, 15))

#ax.scatter(viz_sqft, viz_pr_sf, c=viz_zip, cmap='magma_r')
scatter2 = sns.scatterplot(x="sqft_living", y="pr_sf", data=viz_df, hue='grade', palette='magma_r')

# label axes and title
ax.set_xlabel('Total Square Footage', fontsize=16)
ax.set_ylabel('Price per Square Foot', fontsize=16)
ax.set_title("Price per Square Foot to Total Square Footage\nby Grade of Materials", fontsize=20)

# label and position our legend
plt.legend(title='Grade', loc='upper right', title_fontsize=20);

# save visualization to png
#plt.savefig('images/pr_sf_grade.png');

In [ ]:
# prepare figure
fig, ax = plt.subplots(figsize=(20, 15))

#scatter our data
scatter3 = sns.scatterplot(x="sqft_living", y="price", data=viz_df, hue='zip_rank', palette='magma_r')
#ax.scatter(viz_sqft, viz_target, c=viz_zip, cmap='magma_r')

# label our axes and title
ax.set_xlabel('Total Square Footage', fontsize=16)
ax.set_ylabel('Price in $100,000', fontsize=16)
ax.set_title("Price per Total Square Footage\nby Zip Code Median Value Rank", fontsize=20);

# save visualization to png
#plt.savefig('images/sqft.png');

In [ ]:
# prepare figure
fig, ax = plt.subplots(figsize=(20, 15))

#scatter our data
scatter3 = sns.scatterplot(x="sqft_living", y="pr_sf", data=viz_df, hue='zip_rank', palette='magma_r')
#ax.scatter(viz_sqft, viz_target, c=viz_zip, cmap='magma_r')

# label our axes and title
ax.set_xlabel('Total Square Footage', fontsize=16)
ax.set_ylabel('Price per Square Foot', fontsize=16)
ax.set_title("Price per per Square Foot to Total Square Footage\nby Zip Code Median Value Rank", fontsize=20);

# save visualization to png
#plt.savefig('images/pr_sf_zip.png');

In [ ]:
viz_y = viz_df['price']
viz_x = viz_df.drop('price', axis=1)

fig = plt.figure(figsize=(20,15))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(viz_sqft, viz_grade, viz_target, c=viz_zip, cmap='magma_r')
#ax.scatter(viz_sqft, viz_grade, viz_target, c='red', label="Predictions")
#ax.scatter(viz_sqft, viz_grade, end_z/100000, c='green', label="Actuals")

ax.set_xlabel('Square Feet of Living Space', fontsize=12)
ax.set_ylabel('Grade of Materials', fontsize=12)
ax.set_zlabel('Price', fontsize=12)

ax.set_title("Price per Square Footage and Grade of Materials, by Zip Median Rank", fontsize=20)

# first num is tilt angle, second num is turn angle
# default is about 30,305
# 0, 270 creates side view of pr/sqft
# 0, 360 creates side view of pr/grade
ax.view_init(30, 305)


# save visualization to png
#plt.savefig('images/3d_feats.png');

# APPENDIX

In [ ]:
break

## SVM Parameter Tuning

In [ ]:
# Parameter Tuning

param_grid = {'kernel' : ['linear', 'rbf', 'poly'],
              'gamma' : ['scale', 'auto']            
              }

svr = SVR(C=5, epsilon=.05, tol=.01, verbose=True)
grid_search = GridSearchCV(svr, param_grid, verbose=10, scoring='neg_mean_absolute_error', cv=3, n_jobs=-4)

grid_search.fit(X_train_refined, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

*** FINAL TEST ***
Best parameters set found on train set: 

    {'gamma': 'auto', 'kernel': 'rbf'}

    Grid scores on train set:

    -0.115 (+/-0.002) for {'gamma': 'scale', 'kernel': 'linear'}
    -0.120 (+/-0.002) for {'gamma': 'scale', 'kernel': 'rbf'}
    -0.167 (+/-0.007) for {'gamma': 'scale', 'kernel': 'poly'}
    -0.115 (+/-0.002) for {'gamma': 'auto', 'kernel': 'linear'}
    -0.111 (+/-0.002) for {'gamma': 'auto', 'kernel': 'rbf'}
    -0.239 (+/-0.008) for {'gamma': 'auto', 'kernel': 'poly'}

In [ ]:
# Parameter Tuning

param_grid = {'C' : [3, 5, 7, 10]            
              }

svr = SVR(epsilon=.05, tol=.01, verbose=True, kernel='rbf', gamma='auto')
grid_search = GridSearchCV(svr, param_grid, verbose=10, scoring='neg_mean_absolute_error', cv=3, n_jobs=-4)

grid_search.fit(X_train_refined, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

*** FINAL TEST ***

Best parameters set found on train set: 

    {'C': 10}

    Grid scores on train set:

    -0.111 (+/-0.003) for {'C': 3}
    -0.110 (+/-0.004) for {'C': 5}
    -0.109 (+/-0.003) for {'C': 7}
    -0.109 (+/-0.003) for {'C': 10}

In [ ]:
# Parameter Tuning

param_grid = {'epsilon' : [.01, .05, .1, .5]            
              }

svr = SVR(C=10, tol=.01, verbose=True, kernel='rbf', gamma='auto')
grid_search = GridSearchCV(svr, param_grid, verbose=10, scoring='neg_mean_absolute_error', cv=3, n_jobs=-4)

grid_search.fit(X_train_refined, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

*** FINAL TEST ***

Best parameters set found on train set: 

    {'epsilon': 0.01}

    Grid scores on train set:

    -0.109 (+/-0.003) for {'epsilon': 0.01}
    -0.109 (+/-0.003) for {'epsilon': 0.05}
    -0.110 (+/-0.004) for {'epsilon': 0.1}
    -0.153 (+/-0.004) for {'epsilon': 0.5}

## XGBoost One-Hot Parameter Tuning

In [ ]:
# visualize changes to model score as it is tried on different max depths from 10 to 80, to get a starting point for max depth

from sklearn.model_selection import validation_curve
def ValidationCurve(estimator, predictors, target, param_name, hyperparam):
    
    train_score, test_score = validation_curve(estimator, predictors, target, 
                                               param_name, np.arange(1,33,3), 
                                               cv=5, scoring='neg_mean_absolute_error', n_jobs=-4)
    Rsqaured_train = train_score.mean(axis=1)
    Rsquared_test= test_score.mean(axis=1)
    
    plt.figure(figsize=(10,5))
    plt.plot(np.arange(1,33,3), Rsqaured_train, color='r', linestyle='-', marker='o', label='Training Set')
    plt.plot(np.arange(1,33,3), Rsquared_test, color='b', linestyle='-', marker='x', label='Testing Set')
    plt.legend(labels=['Training Set', 'Testing Set'])
    plt.xlabel(hyperparam)
    plt.ylabel('MAE')
    plt.title("Mean Absolute Error for Max Depth on Train/Test")
    
ValidationCurve(xgb.XGBRegressor(), x_train_boost, y_train, 'max_depth', 'Maximum Depth')

In [ ]:
# Parameter Tuning max_depth

param_grid = {"max_depth": [5, 6, 7, 8],
                          
              }

model = xgb.XGBRegressor(
                 n_estimators=250,                                                                    
                 seed=42,
                 missing=0,
                 eval_metric='mae')

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-4)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'max_depth': 5}

    Grid scores on train set:

    0.881 (+/-0.016) for {'max_depth': 5}
    0.878 (+/-0.020) for {'max_depth': 6}
    0.878 (+/-0.013) for {'max_depth': 7}
    0.872 (+/-0.016) for {'max_depth': 8}

In [ ]:
# Parameter Tuning max_depth and min_child_weight

param_grid = {"min_child_weight" : [6, 8, 10]            
              }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth=5,
                seed=42,
                missing=0,
                eval_metric='mae' )

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-4)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'min_child_weight': 8}

    Grid scores on train set:

    0.885 (+/-0.014) for {'min_child_weight': 6}
    0.885 (+/-0.014) for {'min_child_weight': 8}
    0.884 (+/-0.014) for {'min_child_weight': 10}

In [ ]:
# Parameter Tuning max_depth and min_child_weight

param_grid = {"min_child_weight" : [7, 8, 9]            
              }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth=5,
                seed=42,
                missing=0,
                eval_metric='mae' )

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-4)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'min_child_weight': 8}

    Grid scores on train set:

    0.884 (+/-0.012) for {'min_child_weight': 7}
    0.885 (+/-0.014) for {'min_child_weight': 8}
    0.884 (+/-0.018) for {'min_child_weight': 9}

In [ ]:
# Parameter Tuning gamma

param_grid = {'gamma':[.1, .3, .5, .7, .9],           
              }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 5,
                min_child_weight = 8,
                seed=42,
                missing=0,
                eval_metric='mae' )

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'gamma': 0.1}

    Grid scores on train set:

    0.885 (+/-0.014) for {'gamma': 0.1}
    0.885 (+/-0.014) for {'gamma': 0.3}
    0.885 (+/-0.014) for {'gamma': 0.5}
    0.885 (+/-0.014) for {'gamma': 0.7}
    0.885 (+/-0.014) for {'gamma': 0.9}

In [ ]:
# Parameter Tuning subsample

param_grid = {
 'subsample':[.2, .4, .6, .8, 1],
 
    }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 5,
                min_child_weight = 8,
                gamma = .1,
                seed=42,
                missing=0,
                eval_metric='mae' )

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'subsample': 1}

    Grid scores on train set:

    0.843 (+/-0.022) for {'subsample': 0.2}
    0.869 (+/-0.019) for {'subsample': 0.4}
    0.876 (+/-0.015) for {'subsample': 0.6}
    0.882 (+/-0.019) for {'subsample': 0.8}
    0.885 (+/-0.014) for {'subsample': 1}

In [ ]:
# Parameter Tuning colsample_by_tree

param_grid = {
 'colsample_bytree':[.2, .4, .6, .8, 1]
    }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 5,
                min_child_weight = 8,
                gamma = .1,
                seed=42,
                missing=0,
                eval_metric='mae' )

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'colsample_bytree': 0.8}

    Grid scores on train set:

    0.879 (+/-0.019) for {'colsample_bytree': 0.2}
    0.883 (+/-0.016) for {'colsample_bytree': 0.4}
    0.886 (+/-0.014) for {'colsample_bytree': 0.6}
    0.886 (+/-0.014) for {'colsample_bytree': 0.8}
    0.886 (+/-0.015) for {'colsample_bytree': 1}

In [ ]:
# Parameter Tuning alpha

param_grid = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
    }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 5,
                min_child_weight = 8,
                gamma = .1,
                colsample_bytree=.8,
                seed=42,
                missing=0,
                eval_metric='mae' )

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'reg_alpha': 100}

    Grid scores on train set:

    0.885 (+/-0.014) for {'reg_alpha': 1e-05}
    0.885 (+/-0.014) for {'reg_alpha': 0.01}
    0.885 (+/-0.014) for {'reg_alpha': 0.1}
    0.885 (+/-0.014) for {'reg_alpha': 1}
    0.886 (+/-0.013) for {'reg_alpha': 100}

In [ ]:
# Parameter Tuning alpha

param_grid = {
    'reg_alpha':[75, 100, 125, 150]
    }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 5,
                min_child_weight = 8,
                gamma = .1,
                colsample_bytree=.8,
                seed=42,
                missing=0,
                eval_metric='mae' )

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'reg_alpha': 75}

    Grid scores on train set:

    0.887 (+/-0.014) for {'reg_alpha': 75}
    0.887 (+/-0.014) for {'reg_alpha': 100}
    0.887 (+/-0.014) for {'reg_alpha': 125}
    0.887 (+/-0.014) for {'reg_alpha': 150}

In [ ]:
# Parameter Tuning lambda

param_grid = {'lambda':[0.1, 1, 10, 100, 500]            
              }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 5,
                min_child_weight = 8,
                gamma = .1,
                colsample_bytree=.8,
                reg_alpha = 75,
                seed=42,
                missing=0,
                eval_metric='mae' )

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'lambda': 10}

    Grid scores on train set:

    0.883 (+/-0.017) for {'lambda': 0.1}
    0.887 (+/-0.014) for {'lambda': 1}
    0.887 (+/-0.017) for {'lambda': 10}
    0.885 (+/-0.014) for {'lambda': 100}
    0.874 (+/-0.016) for {'lambda': 500}

In [ ]:
# Parameter Tuning lambda

param_grid = {'lambda':[3, 5, 7, 10, 15]            
              }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 5,
                min_child_weight = 8,
                gamma = .1,
                colsample_bytree=.8,
                reg_alpha = 75,
                seed=42,
                missing=0,
                eval_metric='mae' )

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'lambda': 3}

    Grid scores on train set:

    0.890 (+/-0.012) for {'lambda': 3}
    0.889 (+/-0.013) for {'lambda': 5}
    0.888 (+/-0.018) for {'lambda': 7}
    0.887 (+/-0.017) for {'lambda': 10}
    0.889 (+/-0.018) for {'lambda': 15}

## XGBoost Encoded Parameter Tuning

In [ ]:
# visualize changes to model score as it is tried on different max depths from 10 to 80, to get a starting point for max depth

from sklearn.model_selection import validation_curve
def ValidationCurve(estimator, predictors, target, param_name, hyperparam):
    
    train_score, test_score = validation_curve(estimator, predictors, target, 
                                               param_name, np.arange(1,33,3), 
                                               cv=5, scoring='neg_mean_absolute_error', n_jobs=-4)
    Rsqaured_train = train_score.mean(axis=1)
    Rsquared_test= test_score.mean(axis=1)
    
    plt.figure(figsize=(10,5))
    plt.plot(np.arange(1,33,3), Rsqaured_train, color='r', linestyle='-', marker='o', label='Training Set')
    plt.plot(np.arange(1,33,3), Rsquared_test, color='b', linestyle='-', marker='x', label='Testing Set')
    plt.legend(labels=['Training Set', 'Testing Set'])
    plt.xlabel(hyperparam)
    plt.ylabel('MAE')
    plt.title("Mean Absolute Error for Max Depth on Train/Test")
    
ValidationCurve(xgb.XGBRegressor(), x_train_boost_encoded, y_train, 'max_depth', 'Maximum Depth')

In [ ]:
param_grid = {"max_depth": [2, 4, 6],       
              }

model = xgb.XGBRegressor(
                n_estimators=250,
                seed=42,
                missing=0,
                eval_metric='mae' )
grid_search = GridSearchCV(model, param_grid=param_grid, cv = 3, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost_encoded, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'max_depth': 4}

    Grid scores on train set:

    0.881 (+/-0.008) for {'max_depth': 2}
    0.885 (+/-0.007) for {'max_depth': 4}
    0.876 (+/-0.012) for {'max_depth': 6}

In [ ]:
# Parameter Tuning

param_grid = {"max_depth": [3, 4, 5],        
              }

model = xgb.XGBRegressor(
                n_estimators=250,
                seed=42,
                missing=0,
                eval_metric='mae' )
grid_search = GridSearchCV(model, param_grid=param_grid, cv = 3, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost_encoded, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'max_depth': 3}

    Grid scores on train set:

    0.885 (+/-0.010) for {'max_depth': 3}
    0.885 (+/-0.007) for {'max_depth': 4}
    0.881 (+/-0.007) for {'max_depth': 5}

In [ ]:
# Parameter Tuning

param_grid = {"min_child_weight" : [6, 8, 10, 20],        
              }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 3,
                seed=42,
                missing=0,
                eval_metric='mae' )
grid_search = GridSearchCV(model, param_grid=param_grid, cv = 3, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost_encoded, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'min_child_weight': 20}

    Grid scores on train set:

    0.885 (+/-0.008) for {'min_child_weight': 6}
    0.887 (+/-0.009) for {'min_child_weight': 8}
    0.886 (+/-0.008) for {'min_child_weight': 10}
    0.887 (+/-0.009) for {'min_child_weight': 20}}

In [ ]:
# Parameter Tuning

param_grid = {'gamma': [.2, .4, .5, .6, .8, 1],        
              }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 3,
                min_child_weight=20,
                seed=42,
                missing=0,
                eval_metric='mae' )
grid_search = GridSearchCV(model, param_grid=param_grid, cv = 3, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost_encoded, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'gamma': 0.2}

    Grid scores on train set:

    0.887 (+/-0.009) for {'gamma': 0.2}
    0.887 (+/-0.009) for {'gamma': 0.4}
    0.887 (+/-0.009) for {'gamma': 0.5}
    0.887 (+/-0.009) for {'gamma': 0.6}
    0.887 (+/-0.009) for {'gamma': 0.8}
    0.887 (+/-0.009) for {'gamma': 1}

In [ ]:
# Parameter Tuning

param_grid = {'gamma': [.1, .2, .3],        
              }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 3,
                min_child_weight=20,
                seed=42,
                missing=0,
                eval_metric='mae' )
grid_search = GridSearchCV(model, param_grid=param_grid, cv = 3, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost_encoded, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'gamma': 0.1}

    Grid scores on train set:

    0.887 (+/-0.009) for {'gamma': 0.1}
    0.887 (+/-0.009) for {'gamma': 0.2}
    0.887 (+/-0.009) for {'gamma': 0.3}

In [ ]:
# Parameter Tuning

param_grid = {'subsample':[.6, .7, .8, .9, 1],        
              }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 3,
                min_child_weight=20,
                gamma = .1,
                seed=42,
                missing=0,
                eval_metric='mae' )
grid_search = GridSearchCV(model, param_grid=param_grid, cv = 3, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost_encoded, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'subsample': 1}

    Grid scores on train set:

    0.879 (+/-0.008) for {'subsample': 0.6}
    0.882 (+/-0.007) for {'subsample': 0.7}
    0.883 (+/-0.009) for {'subsample': 0.8}
    0.886 (+/-0.008) for {'subsample': 0.9}
    0.887 (+/-0.009) for {'subsample': 1}

In [ ]:
# Parameter Tuning

param_grid = {'colsample_bytree':[.6, .7, .8, .9, 1],        
              }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 3,
                min_child_weight=20,
                gamma = .1,
                seed=42,
                missing=0,
                eval_metric='mae' )
grid_search = GridSearchCV(model, param_grid=param_grid, cv = 3, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost_encoded, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'colsample_bytree': 0.9}

    Grid scores on train set:

    0.887 (+/-0.009) for {'colsample_bytree': 0.6}
    0.887 (+/-0.008) for {'colsample_bytree': 0.7}
    0.887 (+/-0.008) for {'colsample_bytree': 0.8}
    0.888 (+/-0.008) for {'colsample_bytree': 0.9}
    0.887 (+/-0.010) for {'colsample_bytree': 1}

In [ ]:
# Parameter Tuning

param_grid = {'reg_alpha':[1e-5, 1e-2, 0.1, 1, 10, 100],        
              }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 3,
                min_child_weight=20,
                gamma = .1,
                colsample_bytree=.9,
                seed=42,
                missing=0,
                eval_metric='mae' )
grid_search = GridSearchCV(model, param_grid=param_grid, cv = 3, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost_encoded, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'reg_alpha': 0.1}

    Grid scores on train set:

    0.888 (+/-0.008) for {'reg_alpha': 1e-05}
    0.888 (+/-0.008) for {'reg_alpha': 0.01}
    0.888 (+/-0.008) for {'reg_alpha': 0.1}
    0.888 (+/-0.008) for {'reg_alpha': 1}
    0.888 (+/-0.008) for {'reg_alpha': 10}
    0.888 (+/-0.008) for {'reg_alpha': 100}

In [ ]:
# Parameter Tuning

param_grid = {'reg_alpha':[.05, .1, .15,  .3, .5],        
              }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 3,
                min_child_weight=20,
                gamma = .1,
                colsample_bytree=.9,
                seed=42,
                missing=0,
                eval_metric='mae' )
grid_search = GridSearchCV(model, param_grid=param_grid, cv = 3, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost_encoded, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'reg_alpha': 0.15}

    Grid scores on train set:

    0.888 (+/-0.008) for {'reg_alpha': 0.05}
    0.888 (+/-0.008) for {'reg_alpha': 0.1}
    0.888 (+/-0.008) for {'reg_alpha': 0.15}
    0.888 (+/-0.008) for {'reg_alpha': 0.3}
    0.888 (+/-0.008) for {'reg_alpha': 0.5}

In [ ]:
# Parameter Tuning

param_grid = {'reg_lambda':[1e-5, 1e-2, 0.1, 1, 10, 100],        
              }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 3,
                min_child_weight=20,
                gamma = .1,
                colsample_bytree=.9,
                reg_alpha=.15,
                seed=42,
                missing=0,
                eval_metric='mae' )
grid_search = GridSearchCV(model, param_grid=param_grid, cv = 3, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost_encoded, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'reg_lambda': 1}

    Grid scores on train set:

    0.887 (+/-0.006) for {'reg_lambda': 1e-05}
    0.887 (+/-0.006) for {'reg_lambda': 0.01}
    0.887 (+/-0.006) for {'reg_lambda': 0.1}
    0.888 (+/-0.008) for {'reg_lambda': 1}
    0.888 (+/-0.010) for {'reg_lambda': 10}
    0.885 (+/-0.010) for {'reg_lambda': 100}

In [ ]:
# Parameter Tuning

param_grid = {'reg_lambda':[.5, 1, 1.5, 3],        
              }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 3,
                min_child_weight=20,
                gamma = .1,
                colsample_bytree=.9,
                reg_alpha=.15,
                seed=42,
                missing=0,
                eval_metric='mae' )
grid_search = GridSearchCV(model, param_grid=param_grid, cv = 3, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(X_train_encoded, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'reg_lambda': 3}

    Grid scores on train set:

    0.888 (+/-0.008) for {'reg_lambda': 0.5}
    0.887 (+/-0.008) for {'reg_lambda': 1}
    0.886 (+/-0.010) for {'reg_lambda': 1.5}
    0.888 (+/-0.009) for {'reg_lambda': 3}